In [57]:
spark.stop()

# Set paths

DEVEL_SRC must contain the directory use-cases and pykhaos

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when '-' is removed from name, remove also this line and adapt imports 
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
# AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
# if AMDOCS_SRC not in sys.path: 
#     sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
        
from project.project_generic import Project

import pykhaos.utils.notebooks as nb

import time
start_time = time.time()

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    EXTERNAL_LIB = os.path.join(os.environ.get('BDA_USER_HOME', ''), "lib", "external_libs")
    if EXTERNAL_LIB not in sys.path:
        sys.path.append(EXTERNAL_LIB)
    # feel free from commenting this line and the other ones that begin with "%%notify" if you do not have 
    # the extension installed or copy de lib from /var/SP/data/home/csanc109/lib/external_libs/jupyternotify/
    %load_ext jupyternotify 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

20200121-081836 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20200121_081836.log


<IPython.core.display.Javascript object>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when, desc, asc, to_date, create_map, sum as sql_sum
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract
from itertools import chain
from churn.datapreparation.general.data_loader import get_unlabeled_car, get_port_requests_table, get_numclients_under_analysis
from churn.utils.constants import PORT_TABLE_NAME
from churn.utils.udf_manager import Funct_to_UDF
from pyspark.sql.functions import substring, datediff, row_number
from pykhaos.utils.date_functions import move_date_n_days, move_date_n_cycles
from pykhaos.utils.hdfs_functions import check_hdfs_exists
from pykhaos.modeling.model_performance import get_lift

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Functions

## Start spark context

In [4]:
from churn.utils.general_functions import init_spark
spark = init_spark("navcomp+adobe")
sc = spark.sparkContext

Ended spark session: 23.4550309181 secs | default parallelism=4


# Tasa de evaporacion trigger nav comp

In [ ]:
tr_date_ = "20190925"
filter_ = 'comps'
save_ = True
verbose = True


from churn.analysis.triggers.navcomp.navcomp_utils import get_labeled_set_msisdn
from churn.analysis.triggers.base_utils.base_utils import get_active_filter, get_disconnection_process_filter, get_churn_call_filter

tr_set = get_labeled_set_msisdn(spark, tr_date_, sources='all', save_ = save_, verbose = verbose) 

# Modeling filters

tr_active_filter = get_active_filter(spark, tr_date_, 90)

tr_disconnection_filter = get_disconnection_process_filter(spark, tr_date_, 90)

tr_churn_call_filter = get_churn_call_filter(spark, tr_date_, 90, 'msisdn')

tr_set = tr_set \
    .join(tr_active_filter, ['msisdn'], 'inner') \
    .join(tr_disconnection_filter, ['nif_cliente'], 'inner') \
    .join(tr_churn_call_filter, ['msisdn'], 'inner')

tr_set = filter_population(spark, tr_set, filter_)

tr_set.columns


In [ ]:
[col_ for col_ in tr_set.columns if "port" in col_]

In [ ]:
df_test = tr_set

In [ ]:
#tr_set_churn = tr_set_churn.cache()
tr_set_churn.count()

In [ ]:
competitors = ["PEPEPHONE", "ORANGE", "JAZZTEL", "MOVISTAR", "MASMOVIL", "YOIGO", "VODAFONE", "LOWI", "O2", "unknown"]

def get_most_consulted_operator(consulted_list, days_since_1st_navigation_list):
    if not consulted_list: return "None"
    combined_list = [(a,b,c) for a,b,c in zip(consulted_list, days_since_1st_navigation_list, competitors)]
    # sort by consulted list and then by days since 1st navigation
    sorted_list = sorted(combined_list, key = lambda x : (x[0], x[1]), reverse=True)
    return sorted_list[0][2] if sorted_list else "None"


get_most_consulted_operator_udf = udf(lambda x,y: get_most_consulted_operator(x,y), StringType())


df_test = (df_test
             .withColumn('consulted_list', array(*[col_ + "_sum_count" for col_ in competitors]))
             .withColumn('days_since_1st_navigation_list', array(*[col_ + "_max_days_since_navigation" for col_ in competitors]))
             .withColumn('most_consulted_operator', get_most_consulted_operator_udf(col('consulted_list'), col("days_since_1st_navigation_list"))))

#df_test.select(*([col_ + "_sum_count" for col_ in competitors] + [col_ + "_max_days_since_navigation" for col_ in competitors] +["most_consulted_operator"])).show()

In [ ]:
from churn_nrt.src.data.sopos_dxs import MobPort

df_target = MobPort(spark).get_module(tr_date_)


tr_set_target = tr_set.join(df_target, on=["msisdn"], how="left")
tr_set_churn = tr_set_target.where(col("label_mob")==1)

#tr_set_churn.select('portout_date_mob', "max_days_since_navigation_comps").show()

In [ ]:
# days_until_churn: days since closing_day to churn_event (first churn event)
from pyspark.sql.functions import from_unixtime,unix_timestamp
# days_to_churn = fecha_de_sol_porta – fecha_de_la_última_navegación

# days_until_churn: days since closing_day to churn_event (first churn event)
tr_set_churn = (tr_set_churn.withColumn("days_until_sopo", when(col("portout_date_mob").isNotNull(), 
                                                                 datediff(col("portout_date_mob"),
                                                                          from_unixtime(unix_timestamp(lit(tr_date_), "yyyyMMdd")))).otherwise(-1)))

tr_set_churn = tr_set_churn.withColumn("days_until_churn", col("days_until_sopo") + col("min_days_since_navigation_comps"))



In [ ]:
col_ = "days_until_churn"
n = 20

bins, counts = tr_set_churn.select(col_).rdd.flatMap(lambda x: x).histogram(n)

plt.figure()
plt.hist(bins[:-1], bins=bins, weights=counts)
plt.xlabel("** {} **".format(col_))

total = tr_set_churn.count()

import numpy as np
counts_cum = np.cumsum(counts)
counts_cum = [100.0 * cc/total for cc in counts_cum]

plt.figure()
plt.hist(bins[:-1], bins=bins, weights=counts_cum)
plt.grid(True)
plt.yticks(list(range(10,110,10))) 
plt.xlabel("cumsum {}".format(col_))
plt.ylabel("% churners [total churners = {}]".format(total))

pd.DataFrame({"bins" : bins[1:], "counts" : counts_cum})

In [ ]:
base_df.unpersist()

from churn.analysis.triggers.base_utils.base_utils import get_customer_base
date_ = "20190930"
add_columns_customer = ["birth_date", "fecha_naci", 'CLASE_CLI_COD_CLASE_CLIENTE', 'X_CLIENTE_PRUEBA', "TIPO_DOCUMENTO"]
add_columns=["TARIFF", 'COD_ESTADO_GENERAL', "srv_basic"]
closing_day = date_
date_ = date_
base_df = (get_customer_base(spark, date_, add_columns=add_columns, 
                             add_columns_customer=add_columns_customer,
                            active_filter=False).filter(col('rgu')=='mobile'))#.select(*(['msisdn', 'nif_cliente', 'num_cliente'] + 
                                                                                                                                           #add_columns + add_columns_customer)))

print(base_df.columns)

base_df = (base_df.withColumn("birth_date", when( (col("birth_date").isNull() | (col("birth_date") == 1753)), col("fecha_naci")).otherwise(col("birth_date")).cast(IntegerType()))
                  .withColumn("age", when(col("birth_date") != 1753, lit(int(closing_day[:4])) - col("birth_date")).otherwise(-1))
          ).drop("birth_date")

base_df = base_df.cache()
base_df.columns

In [ ]:
base_df.groupby("srv_basic").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.groupby("CLASE_CLI_COD_CLASE_CLIENTE").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.select("X_CLIENTE_PRUEBA").dtypes

In [ ]:
base_df.groupby("X_CLIENTE_PRUEBA").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.groupby("COD_ESTADO_GENERAL").agg(*[sql_count("*").alias("count")]).show()


In [ ]:
base_df.groupby("TARIFF").agg(*[sql_count("*").alias("count")]).show(100)


In [ ]:
base_df.select("NIF_CLIENTE").distinct().count()

In [ ]:
base_df.select("NUM_CLIENTE").distinct().count()


In [ ]:
base_df.select("msisdn").distinct().count()

In [ ]:
base_df.count()

In [ ]:
spark.read.load("/user/csanc109/data/mobile_base_20190930_parquet").count()

In [ ]:
base_df_insights.coalesce(1).write.mode('overwrite').save("/user/csanc109/data/mobile_base_20190930_parquet")

In [ ]:
base_df_insights = base_df.select("msisdn", "num_cliente", "nif_cliente", "srv_basic", "rgu", 'CLASE_CLI_COD_CLASE_CLIENTE', 'COD_ESTADO_GENERAL', "TARIFF")
base_df_insights.coalesce(1).write.mode('overwrite').format('csv').option('sep', '|').option('header', 'true').save("/user/csanc109/data/mobile_base_20190930")


In [ ]:
base_df  = base_df.withColumn("age_disc",  when(col("age").isNull(), "other")
                                         .when(col("age")<20, "<20")
                                         .when( (col("age")>=20) & (col("age")<25) , "[20-25)")
                                         .when( (col("age")>=25) & (col("age")<30) , "[25-30)")
                                         .when( (col("age")>=30) & (col("age")<35) , "[30-35)")
                                         .when( (col("age")>=35) & (col("age")<40) , "[35-40)")
                                         .when( (col("age")>=40) & (col("age")<45) , "[40-45)")
                                         .when( (col("age")>=45) & (col("age")<50) , "[45-50)")
                                         .when( (col("age")>=50) & (col("age")<55) , "[50-55)")
                                         .when( (col("age")>=55) & (col("age")<60) , "[55-60)")
                                         .when( (col("age")>=60) & (col("age")<65) , "[60-65)")
                                         .when( col("age")>=65, ">=65")
                                         .otherwise("other"))

base_df.groupby("age_disc").agg(sql_count("*").alias("count")).show()


In [ ]:
df_tgs.count()

In [ ]:
closing_day = "20191031"
df_tgs = spark.read.load("/data/udf/vf_es/churn/extra_feats_mod/tgs/year={}/month={}/day={}".format(int(closing_day[:4]), int(closing_day[4:6]), int(closing_day[6:])))
df_tgs.select('tgs_days_since_f_inicio_bi',
 'tgs_days_since_f_inicio_bi_exp',
 'tgs_days_until_f_fin_bi',
 'tgs_days_until_f_fin_bi_exp',
 'tgs_days_until_fecha_fin_dto').show()

In [ ]:
df_tgs.columns

In [ ]:


df_tgs = df_tgs.withColumn("blindaje_categ", when(  col("tgs_days_until_f_fin_bi")>60, "blindado")
                                      .when( (col("tgs_days_until_f_fin_bi")>0) & (col("tgs_days_until_f_fin_bi")<=60), "blindado proximo")
                                      .when( ((col("tgs_blindaje_bi_expirado")==1) & (col("tgs_days_since_f_inicio_bi_exp")>60)), "desblindado")
                                      .when( ((col("tgs_blindaje_bi_expirado")==1) & (col("tgs_days_since_f_inicio_bi_exp")>=0) & (col("tgs_days_since_f_inicio_bi_exp")<=60)), "desblindado reciente")
                    )

df_tgs.groupby("blindaje_categ").agg(sql_count("*").alias("count")).show()


In [ ]:

from churn.analysis.triggers.navcomp.navcomp_utils import get_latest_scores

df_scores = get_latest_scores(closing_day)

mean_score = df_scores.select(sql_avg('scoring').alias('mean_score')).rdd.first()['mean_score']


mean_score

# TESTING NEW ATTRIBUTES

In [ ]:
tr_date_ = "20191030"
filter_ = 'comps'
save_ = True
verbose = True


from churn.analysis.triggers.navcomp.navcomp_utils import get_labeled_set_msisdn
from churn.analysis.triggers.base_utils.base_utils import get_active_filter, get_disconnection_process_filter, get_churn_call_filter

tr_set = get_labeled_set_msisdn(spark, tr_date_, sources='all', save_ = save_, verbose = verbose) 
tr_set.columns

In [ ]:

def _get_customer_master_feats(customer_tr_df):

    class UDFclass:

        @staticmethod
        def compute_rgus_attrib(current_list, rgu_name):
            import collections
            rgus_dict = collections.Counter(current_list) if current_list is not None else {}
            if rgu_name not in rgus_dict:
                return 0
            else:
                return rgus_dict[rgu_name]

    compute_rgus_attrib_udf = udf(lambda x, y: UDFclass.compute_rgus_attrib(x, y), IntegerType())

    customer_tr_df = customer_tr_df.withColumn("nb_rgus_mobile", compute_rgus_attrib_udf("rgus_list", lit("mobile")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_tv", compute_rgus_attrib_udf("rgus_list", lit("tv")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_fixed", compute_rgus_attrib_udf("rgus_list", lit("fixed")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_fbb", compute_rgus_attrib_udf("rgus_list", lit("fbb")))
    customer_tr_df = customer_tr_df.withColumn("nb_rgus_bam", compute_rgus_attrib_udf("rgus_list", lit("bam")))

    return customer_tr_df


In [ ]:
date_ = "20191022"

from churn.analysis.triggers.navcomp.metadata import get_metadata

from churn.analysis.triggers.base_utils.base_utils import get_customer_base
add_columns_customer = ["birth_date", "fecha_naci", 'CLASE_CLI_COD_CLASE_CLIENTE', 'X_CLIENTE_PRUEBA', "TIPO_DOCUMENTO"]
#add_columns = ["TARIFF", 'COD_ESTADO_GENERAL', "srv_basic"]

base_df = get_customer_base(spark, date_, add_columns_customer=add_columns_customer).filter(col('rgu')=='mobile').select('msisdn', 'nif_cliente', 'num_cliente', 'birth_date', 'fecha_naci')
base_df = base_df.drop_duplicates(['msisdn', 'nif_cliente', 'num_cliente'])
base_df = (base_df.withColumn("birth_date", when((col("birth_date").isNull() | (col("birth_date") == 1753)), col("fecha_naci")).otherwise(col("birth_date")).cast(IntegerType()))
                  .withColumn("age", when(col("birth_date") != 1753, lit(int(date_[:4])) - col("birth_date")).otherwise(-1)))
base_df = base_df.withColumn("age_disc", when(col("age").isNull(), "other").when(col("age") < 20, "<20")
                                        .when((col("age") >= 20) & (col("age") < 25), "[20-25)")
                                        .when((col("age") >= 25) & (col("age") < 30), "[25-30)")
                                        .when((col("age") >= 30) & (col("age") < 35), "[30-35)")
                                        .when((col("age") >= 35) & (col("age") < 40), "[35-40)")
                                        .when((col("age") >= 40) & (col("age") < 45), "[40-45)")
                                        .when((col("age") >= 45) & (col("age") < 50), "[45-50)")
                                        .when((col("age") >= 50) & (col("age") < 55), "[50-55)")
                                        .when((col("age") >= 55) & (col("age") < 60), "[55-60)")
                                        .when((col("age") >= 60) & (col("age") < 65), "[60-65)")
                                        .when(col("age") >= 65, ">=65").otherwise("other")).drop("birth_date")
# base_df.columns ['msisdn', 'nif_cliente', 'num_cliente', 'fecha_naci', 'age', 'age_disc']

from churn.analysis.triggers.orders.customer_master import get_customer_master

customer_metadata = get_metadata(spark, sources=['customer'])

customer_feats = customer_metadata.select('feature').rdd.map(lambda x: x['feature']).collect() + ['nif_cliente']



customer_tr_df = (get_customer_master(spark, date_, unlabeled=True)
                        .filter((col('segment_nif') != 'Pure_prepaid') & (col("segment_nif").isNotNull())).drop_duplicates(["nif_cliente"])
                )

customer_tr_df = (_get_customer_master_feats(customer_tr_df).select('nif_cliente', 
 'segment_nif',
 'nb_rgus',
 'rgus_list',
 'tgs_days_until_fecha_fin_dto',
 'nb_rgus_mobile',
 'nb_rgus_tv',
 'nb_rgus_fixed',
 'nb_rgus_fbb',
 'nb_rgus_bam'))

customer_map_tmp = customer_metadata.select("feature", "imp_value", "type").rdd.map(lambda x: (x["feature"], x["imp_value"], x["type"])).collect()

customer_map = dict([(x[0], str(x[1])) if x[2] == "categorical" else (x[0], float(x[1])) for x in customer_map_tmp])

base_df = base_df.join(customer_tr_df, ['nif_cliente'], 'inner').na.fill(customer_map)

base_df = base_df.select(customer_feats).na.fill(customer_map)



In [ ]:
base_df.count()

In [ ]:
spark.read.load("/data/udf/vf_es/churn/triggers/navcomp_msisdn_data/year=2019/month=11/day=7").columns

In [ ]:
df_mod_navcomp = spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=triggers_navcomp/year=2019")
df_mod_navcomp.select("year", "month", "day", "predict_closing_date").distinct().show()

In [ ]:
customer_tr_df.select("msisdn", 'nb_rgus', 'rgus_list', 'tgs_days_until_fecha_fin_dto', 'nb_rgus_mobile', 'nb_rgus_tv', 'nb_rgus_fixed', 'nb_rgus_fbb', 'nb_rgus_bam')

In [ ]:
tt_date = "20190601"
from pykhaos.utils.date_functions import move_date_n_days
end_port = move_date_n_days(tt_date, 15)
from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests
target = get_mobile_portout_requests(spark, tt_date, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label', "portout_date_mob")
target.columns

# VANISHING RATE DEL TOP-k

In [ ]:
spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests/year=2019/month=11/day=7").columns

In [ ]:
[col_ for col_ in spark.read.load("/data/udf/vf_es/churn/triggers/navcomp_msisdn_data/year=2019/month=10/day=24").columns if "age" in col_]

In [ ]:
navcomp_date = "20190921"
target_date = navcomp_date

tt_date = -1 # "20190601"

navcomp_tr_df = spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests/year={}/month={}/day={}".format(int(navcomp_date[:4]), int(navcomp_date[4:6]), int(navcomp_date[6:]))).drop("label")

# Labeling

from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests

from pykhaos.utils.date_functions import move_date_n_days

end_port = move_date_n_days(target_date, 15)

target = get_mobile_portout_requests(spark, navcomp_date, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label', "portout_date_mob")

# Modeling filters

tt_active_filter = get_active_filter(spark, navcomp_date, 90)

tt_disconnection_filter = get_disconnection_process_filter(spark, navcomp_date, 90)

tt_churn_call_filter = get_churn_call_filter(spark, navcomp_date, 90, 'msisdn')


In [ ]:
tt_set.count()

In [ ]:
from churn.analysis.triggers.navcomp.navcomp_model import filter_population

tt_set = navcomp_tr_df \
    .join(tt_active_filter, ['msisdn'], 'inner') \
    .join(tt_disconnection_filter, ['nif_cliente'], 'inner') \
    .join(tt_churn_call_filter, ['msisdn'], 'inner')

tt_set = filter_population(spark, tt_set, filter_)

tr_set_target = tt_set.join(target, ['msisdn'], 'left').na.fill({'label': 0.0})

print "[Info navcomp_model_production] After all the filters - Sie of the test set: " + str(tt_set.count()) + " - Number of distinct MSISDNs in the test set: " + str(tt_set.select('msisdn').distinct().count())

tr_set_churn = tr_set_target.where(col("label")==1)

from pyspark.sql.functions import from_unixtime,unix_timestamp


In [ ]:
from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests


df_predictions = spark.read.load("/data/udf/vf_es/churn/triggers/nav_comp_tests_nav_comp_tests_alllabels/year=2019/month=9/day=21/")

target_date = "20190921"
navcomp_date = target_date

from pykhaos.utils.date_functions import move_date_n_days

end_port = move_date_n_days(target_date, 15)

target = get_mobile_portout_requests(spark, target_date, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label', "portout_date_mob")

tr_set_target = df_predictions.drop("label").join(target, ['msisdn'], 'left').na.fill({'label': 0.0})


In [ ]:
import math
num_nile = int(math.floor(float(tr_set_churn.count()) / 100.0))

lift = get_lift(tr_set_churn, 'scoring', 'label', num_nile, 1.0)



In [ ]:
[col_ for col_ in tr_set_target.columns if "scor" in col_]

In [ ]:
import pandas as pd
from pyspark.sql.functions import from_unixtime,unix_timestamp

col_ = "days_until_churn"
n = 20

myschema = tr_set_target.schema

total = tr_set_target.count()
print("total", total)

for top_ in [2000,3000,5000, 10000, 20000]:
    
    print("************** Analyzing top_{}".format(top_))
    
    tr_set_target = tr_set_target.sort(desc("model_score"))
    
    df_top = spark.createDataFrame(tr_set_target.head(top_), schema=myschema)
    
    tr_set_churn = df_top.where(col("label")==1)
    num_churners = tr_set_churn.count()
    
    tr_set_churn = (tr_set_churn.withColumn("days_until_sopo", when(col("portout_date_mob").isNotNull(), 
                                                                 datediff(col("portout_date_mob"), from_unixtime(unix_timestamp(lit(navcomp_date), "yyyyMMdd")))).otherwise(-1)))

    tr_set_churn = tr_set_churn.withColumn("days_until_churn", col("days_until_sopo") + col("min_days_since_navigation_comps"))

    print("1) num scores nulos", tr_set_churn.where(col("model_score").isNull()).count())

    tr_set_churn = tr_set_churn.where(col("model_score").isNotNull())
                                                
    tr_set_churn = tr_set_churn.cache()

    print("2) churn_rate top{} = {}".format(top_, 100.0*num_churners/top_))
    
    
    bins, counts = tr_set_churn.select(col_).rdd.flatMap(lambda x: x).histogram(n)

    plt.figure()
    plt.hist(bins[:-1], bins=bins, weights=counts)
    plt.xlabel("** {} ** (top = {}) (num_churners={}) (churn_rate={})".format(col_, top_, num_churners, 100.0*num_churners/top_))

    import numpy as np
    counts_cum = np.cumsum(counts)
    counts_cum = [100.0 * cc/num_churners for cc in counts_cum]
    
    plt.figure()
    plt.hist(bins[:-1], bins=bins, weights=counts_cum)
    plt.grid(True)
    plt.yticks(list(range(10,110,10))) 
    plt.xlabel("cumsum {} (top = {}) (num_churners={}) (churn_rate={})".format(col_, top_, num_churners, 100.0*num_churners/top_))

    print(top_, pd.DataFrame({"bins" : bins[1:], "counts" : counts_cum}))
    

In [ ]:
from churn.datapreparation.general.tgs_data_loader import get_tgs
from churn_nrt.src.utils.date_functions import get_previous_cycle

closing_day_tgs = get_previous_cycle(closing_day)
print("Getting tgs for date {}".format(closing_day_tgs))

df_tgs = get_tgs(spark, closing_day_tgs, closing_day_tgs[:6], impute_nulls=False).select("nif_cliente", 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi', 'msisdn')
#df_tgs.columns -- ['nif_cliente', 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi']


df_tgs = df_tgs.withColumn("blindaje_disc", when(col("tgs_days_until_f_fin_bi").isNull(), "none")
                             .when(col("tgs_days_until_f_fin_bi") > 60, "hard")
                             .when((col("tgs_days_until_f_fin_bi") >   0) & (col("tgs_days_until_f_fin_bi") <= 60), "soft")
                             .when((col("tgs_days_until_f_fin_bi") <=  0) & (col("tgs_days_until_f_fin_bi") >= -60), "soft-nobounded")
                             .otherwise("none"))

#df_tgs.groupby('tgs_days_until_f_fin_bi', "blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
closing_day = "20191016"
date_ = closing_day
#from churn.analysis.triggers.orders.customer_master import get_tgs
from churn.datapreparation.general.tgs_data_loader import get_tgs
from churn_nrt.src.utils.date_functions import get_previous_cycle

closing_day_tgs = get_previous_cycle(closing_day)
print("Getting tgs for date {}".format(closing_day_tgs))

df_tgs = get_tgs(spark, closing_day_tgs, closing_day_tgs[:6], impute_nulls=False).select("nif_cliente", 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi', 'msisdn')
#df_tgs.columns -- ['nif_cliente', 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi']


df_tgs = df_tgs.withColumn("blindaje_disc", when(col("tgs_days_until_f_fin_bi").isNull(), "none")
                             .when(col("tgs_days_until_f_fin_bi") > 60, "hard")
                             .when((col("tgs_days_until_f_fin_bi") >   0) & (col("tgs_days_until_f_fin_bi") <= 60), "soft")
                             .when((col("tgs_days_until_f_fin_bi") <=  0) & (col("tgs_days_until_f_fin_bi") >= -60), "soft-nobounded")
                             .otherwise("none"))

# Desblindado [fcarren: Cliente desblindado donde ese desblindaje se produjo hace más de 2 meses, o que nunca tuvo blindaje]
# Desblindado reciente [fcarren: Cliente desblindado, pero su desblindaje se ha producido en los recientes 1 o 2 meses]
# Blindado próximo [fcarren: Cliente blindado, pero su le quedan sólo  1 o 2 meses para que se desblinde]
# Blindado [fcarren: Cliente blindado, con blindaje activo de más de 2 meses]



In [ ]:
df_tgs.groupby("blindaje_disc", "tgs_days_until_f_fin_bi").agg(sql_count("*").alias("count")).sort(asc("tgs_days_until_f_fin_bi")).show(300)

In [ ]:
df_tgs_raw.groupby("blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
from churn.analysis.triggers.navcomp.navcomp_utils import get_navcomp_car_msisdn

df_navcomp = get_navcomp_car_msisdn(spark, date_, sources=["navcomp","customer","ccc","spinners","scores"], save_ = False, verbose = False)
nb_rgus_col = [col_ for col_ in df_navcomp.columns if col_.startswith("nb_rgus_")]

filter_ = ['comps']

from churn.analysis.triggers.base_utils.base_utils import get_active_filter, get_disconnection_process_filter, get_churn_call_filter


tt_active_filter = get_active_filter(spark, date_, 90)

tt_disconnection_filter = get_disconnection_process_filter(spark, date_, 90)

tt_churn_call_filter = get_churn_call_filter(spark, date_, 90, 'msisdn')

from churn.analysis.triggers.navcomp.navcomp_model import filter_population

tt_set = df_navcomp \
    .join(tt_active_filter, ['msisdn'], 'inner') \
    .join(tt_disconnection_filter, ['nif_cliente'], 'inner') \
    .join(tt_churn_call_filter, ['msisdn'], 'inner')




In [ ]:
#from churn.analysis.triggers.navcomp.navcomp_utils import get_volumen_attributes
#df_get_volumen = get_volumen_attributes(spark, date_, nDays=-14)#.select(volumen_feats)


In [ ]:
filter_ = 'comps'

tt_set = filter_population(spark, tt_set, filter_)

#tr_set_target = tt_set.join(target, ['msisdn'], 'left').na.fill({'label': 0.0})

In [ ]:
from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests

from pykhaos.utils.date_functions import move_date_n_days
end_port = move_date_n_days(date_, 15)
target = get_mobile_portout_requests(spark, date_, end_port).withColumnRenamed('label_mob', 'label').select('msisdn', 'label').withColumnRenamed("label", "label_1_15")

from pykhaos.utils.date_functions import move_date_n_days
end_port_2 = move_date_n_days(end_port, 15)
target_2 = get_mobile_portout_requests(spark, end_port, end_port_2).withColumnRenamed('label_mob', 'label').select('msisdn', 'label').withColumnRenamed("label", "label_16_30")


from pykhaos.utils.date_functions import move_date_n_days
end_port_3 = move_date_n_days(date_, 30)
target_3 = get_mobile_portout_requests(spark, date_, end_port_3).withColumnRenamed('label_mob', 'label').select('msisdn', 'label').withColumnRenamed("label", "label_1_30")


In [ ]:
date_,end_port,end_port_2, end_port_3

In [ ]:
from churn.analysis.triggers.orders.customer_master import get_customer_master
customer_tr_df = (get_customer_master(spark, date_, unlabeled=True).filter((col('segment_nif') != 'Pure_prepaid') & (col("segment_nif").isNotNull())).drop_duplicates(["nif_cliente"]))


In [ ]:
from churn.analysis.triggers.navcomp.navcomp_utils import get_latest_scores
df_scores = get_latest_scores(spark, date_)

In [ ]:
df_navcomp_select = tt_set.select("most_consulted_operator", "JAZZTEL_max_count", "VODAFONE_sum_count", "PEPEPHONE_min_days_since_navigation", "VODAFONE_min_days_since_navigation", 
                                      "norm_min_days_since_navigation_comps", "MOVISTAR_distinct_days_with_navigation", "unknown_distinct_days_with_navigation", "MOVISTAR_sum_count", 
                                      "norm_max_days_since_navigation_comps","msisdn", "age", "age_disc")

In [ ]:
#df_navcomp_select = df_navcomp_select.cache()
df_analysis.count()

In [ ]:
df_analysis = df_tgs.select(*['nif_cliente', 'tgs_days_until_fecha_fin_dto', 'tgs_days_until_f_fin_bi', 'blindaje_disc', 'msisdn']).join(df_navcomp_select, on=["msisdn"], how="inner")
df_analysis = df_analysis.join(df_scores, on=["msisdn"], how="left")
df_analysis = df_analysis.join(customer_tr_df.select("nif_cliente", "segment_nif"), ['nif_cliente'], 'inner')

df_analysis = df_analysis.join(target, ['msisdn'], 'left').na.fill({'label_1_15': 0.0})
df_analysis = df_analysis.join(target_2, ['msisdn'], 'left').na.fill({'label_16_30': 0.0})
df_analysis = df_analysis.join(target_3, ['msisdn'], 'left').na.fill({'label_1_30': 0.0})
#df_analysis.count()

In [ ]:
df_analysis.groupby("blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
df_analysis = df_analysis.withColumn("age_disc_simple", when(col("age").isNull(), "other")
                             .when((col("age") >   0) & (col("age") < 20), "<20")
                             .when((col("age") >= 20) & (col("age") < 35), "[20-35)")
                             .when((col("age") >= 35) & (col("age") < 55), "[35-55)")
                             .when((col("age") >= 55) & (col("age") < 70), "[55-70)")
                             .when(col("age") >= 70, ">=70").otherwise("other"))

operadores = ['PEPEPHONE', 'JAZZTEL', 'MOVISTAR', 'MASMOVIL', 'YOIGO', 'VODAFONE', 'LOWI', 'O2', 'ORANGE']


for op in operadores:
    df_analysis = df_analysis.withColumn("flag_{}".format(op), when(col("most_consulted_operator")==op, 1).otherwise(0))

    
df_analysis = df_analysis.withColumn("most_consulted_operator_simple", when(col("most_consulted_operator").isin(["MOVISTAR", "VODAFONE", "YOIGO"]), col("most_consulted_operator")).otherwise("OTHERS"))


In [ ]:
from pyspark.sql.functions import avg as sql_avg
myschema = df_analysis.schema
df_analysis = df_analysis.sort(desc("scoring"))
df_top20k = spark.createDataFrame(df_analysis.head(20000), schema=myschema)



In [ ]:
df_top20k.groupby("segment_nif").agg(sql_count("*").alias("count")).show()

In [ ]:
df_top20k.groupby(['blindaje_disc']).agg(*([sql_count("*").alias("count"), sql_sum("label_1_15").alias("num_churners_1_15"),                  
                                                                                                             sql_avg("scoring").alias("avg_scoring")])).show(100,truncate=False)

In [ ]:
df_top20k.groupby("blindaje_disc").agg(sql_count("*").alias("count")).show()

In [ ]:
df_top20k.groupby(['age_disc', 'blindaje_disc', "most_consulted_operator", "segment_nif"]).agg(*([sql_count("*").alias("count"), sql_sum("label_1_15").alias("num_churners_1_15"),                  
                                                                                                             sql_avg("scoring").alias("avg_scoring")])).show(2500,truncate=False) 
                                                                                                            

In [ ]:

df_agg_top20k = df_top20k.groupby(['blindaje_disc', "most_consulted_operator", "segment_nif"]).agg(*([sql_count("*").alias("count"), 
                                                                                                             sql_sum("label_1_15").alias("num_churners_1_15"), 
                                                                                                             sql_sum("label_16_30").alias("num_churners_16_30"), 
                                                                                                             sql_sum("label_1_30").alias("num_churners_1_30"), 
                                                                                                             sql_avg("scoring").alias("avg_scoring")] + 
                                                                                                            [sql_sum("flag_{}".format(op)).alias("num_op_{}".format(op)) for op in operadores]))
df_agg_top20k.sort(desc("count")).show(5000, truncate=False)

In [ ]:
df_top20k.groupby(['age_disc_simple', "blindaje_disc"]).agg(*[sql_max("JAZZTEL_max_count").alias("MAX_JAZZTEL_max_count"),
                                                              sql_min("JAZZTEL_max_count").alias("MIN_JAZZTEL_max_count"), 
                                                              sql_avg("JAZZTEL_max_count").alias("AVG_JAZZTEL_max_count"), 
                                                              
                                                              sql_sum("VODAFONE_sum_count").alias("SUM_VODAFONE_sum_count"),
                                                              sql_avg("VODAFONE_sum_count").alias("AVG_VODAFONE_sum_count"),
                                                              sql_min("VODAFONE_sum_count").alias("MIN_VODAFONE_sum_count"),
                                                              sql_max("VODAFONE_sum_count").alias("MAX_VODAFONE_sum_count"),
                                                              
                                                              sql_min("PEPEPHONE_min_days_since_navigation").alias("MIN_PEPEPHONE_min_days_since_navigation"),
                                                              sql_max("PEPEPHONE_min_days_since_navigation").alias("MAX_PEPEPHONE_min_days_since_navigation"),
                                                              sql_avg("PEPEPHONE_min_days_since_navigation").alias("AVG_PEPEPHONE_min_days_since_navigation"),

                                                              sql_min("VODAFONE_min_days_since_navigation").alias("MIN_VODAFONE_min_days_since_navigation"),
                                                              sql_max("VODAFONE_min_days_since_navigation").alias("MAX_VODAFONE_min_days_since_navigation"),
                                                              sql_avg("VODAFONE_min_days_since_navigation").alias("AVG_VODAFONE_min_days_since_navigation"),

                                                              sql_min("norm_min_days_since_navigation_comps").alias("MIN_norm_min_days_since_navigation_comps"),
                                                              sql_max("norm_min_days_since_navigation_comps").alias("MAX_norm_min_days_since_navigation_comps"),
                                                              sql_count("norm_min_days_since_navigation_comps").alias("COUNT_norm_min_days_since_navigation_comps"),

                                                              sql_avg("MOVISTAR_distinct_days_with_navigation").alias("AVG_MOVISTAR_distinct_days_with_navigation"),
                                                              sql_min("MOVISTAR_distinct_days_with_navigation").alias("MIN_MOVISTAR_distinct_days_with_navigation"),
                                                              sql_max("MOVISTAR_distinct_days_with_navigation").alias("MAX_MOVISTAR_distinct_days_with_navigation"),

                                                              sql_avg("unknown_distinct_days_with_navigation").alias("AVG_unknown_distinct_days_with_navigation"),
                                                              sql_min("unknown_distinct_days_with_navigation").alias("MIN_unknown_distinct_days_with_navigation"),
                                                              sql_max("unknown_distinct_days_with_navigation").alias("MAX_unknown_distinct_days_with_navigation"),
                                                              
                                                              sql_avg("MOVISTAR_sum_count").alias("AVG_MOVISTAR_sum_count"),
                                                              sql_min("MOVISTAR_sum_count").alias("MIN_MOVISTAR_sum_count"),
                                                              sql_max("MOVISTAR_sum_count").alias("MAX_MOVISTAR_sum_count"),

                                                              sql_max("norm_max_days_since_navigation_comps").alias("MAX_norm_max_days_since_navigation_comps"),
                                                              sql_min("norm_max_days_since_navigation_comps").alias("MIN_norm_max_days_since_navigation_comps"),
                                                              sql_avg("norm_max_days_since_navigation_comps").alias("AVG_norm_max_days_since_navigation_comps"),

                                                              sql_count("*").alias("count"), 
                                                                                                                          
                                                             ]).show()

In [ ]:
df_agg_2 = df_top20k.groupby(['blindaje_disc', "most_consulted_operator"]).agg(*[sql_count("*").alias("count"), sql_avg("scoring").alias("avg_scoring")])

In [ ]:
df_agg_2.sort(desc("count")).show(50, truncate=False)

In [ ]:
df_agg_3 = df_analysis.groupby(["most_consulted_operator", 'age_disc_simple']).agg(*[sql_count("*").alias("count"), sql_avg("scoring").alias("avg_scoring")])

In [ ]:
df_agg_3.sort(desc("count")).show(150, truncate=False)

In [ ]:
df_model_scores = spark.read.load("/user/csanc109/data/mobile_base_20190930_parquet")
df_model_scores.columns

In [ ]:
df_model_scores2.columns

In [ ]:
df_model_scores2 = spark.read.option("delimiter", "|").option("header",True).csv("/user/csanc109/data/mobile_base_20190930_insigths_campos/mobile_base_20190930_insights_campos.csv")
print(len(df_model_scores2.columns), df_model_scores2.count())

In [ ]:
df_model_scores = spark.read.option("delimiter", "|").option("header",True).csv("/user/csanc109/data/mobile_base_20190930_insigths/mobile_base_20190930_insights.csv")
print(len(df_model_scores.columns), df_model_scores.count())

In [ ]:
# df_insi = spark.read.option("delimiter", "|").option("header",True).csv("/user/csanc109/data/mobile_base_20190930_insigths/mobile_base_20190930_insights.csv")
# # for col_ in df_insi.columns:
# #     if col_ == "MSISDN":
# #         df_insi = df_insi.withColumnRenamed(col_, col_.lower())
# #     else:
# #         df_insi = df_insi.withColumnRenamed(col_, col_.lower()+"_insi")
     
df_model_scores = spark.read.option("delimiter", "|").option("header",True).csv("/user/csanc109/data/mobile_base_20190930_insigths/mobile_base_20190930_insights.csv")


#df_model_scores = spark.read.load("/user/csanc109/data/mobile_base_20190930_parquet")
df_model_scores = df_model_scores.withColumn("scoring", lit(0.0))
df_model_scores = df_model_scores.withColumnRenamed("NIF_CLIENTE", "nif")
df_model_scores = df_model_scores.withColumnRenamed("NUM_CLIENTE", "client_id")
df_model_scores = df_model_scores.withColumnRenamed("MSISDN", "msisdn")

   
df_model_scores.columns

# Create the columns with missing identifiers
for col_ in  ALL_ID_COLS:
    if not col_ in df.columns:
        # Avoid this problem "Caused by: java.lang.RuntimeException: Unsupported data type NullType."
        df = df.withColumn(col_, lit(""))
        print("Added None to non existing col {}".format(col_))


In [ ]:

df_model_scores = spark.read.option("delimiter", "|").option("header",True).csv("/user/csanc109/data/mobile_base_20190930_insigths/mobile_base_20190930_insights.csv")


#df_model_scores = spark.read.load("/user/csanc109/data/mobile_base_20190930_parquet")
df_model_scores = df_model_scores.withColumn("scoring", lit(0.0))
df_model_scores = df_model_scores.withColumnRenamed("NIF_CLIENTE", "nif")
df_model_scores = df_model_scores.withColumnRenamed("NUM_CLIENTE", "client_id")
df_model_scores = df_model_scores.withColumnRenamed("MSISDN", "msisdn")


from pyspark.sql.functions import length, concat_ws, regexp_replace, split
from churn.datapreparation.general.model_outputs_manager import ensure_types_model_scores_columns

EXTRA_INFO_COLS = ['msisdn',"client_id", "nif"]

MODEL_OUTPUTS_NULL_TAG = ""

executed_at = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S ")

return_feed_execution = "20190930"

day_partition = int(return_feed_execution[6:])
month_partition = int(return_feed_execution[4:6])
year_partition = int(return_feed_execution[:4])
training_closing_date = "20190930"

print("Going to insert with partition {} {} {} ".format(year_partition, month_partition, day_partition))

'''
MODEL PARAMETERS
'''

import pandas as pd
df_pandas = pd.DataFrame({
    "model_name": ["mobile_base"],
    "executed_at": [executed_at],
    "model_level": ["msisdn"],
    "training_closing_date": [training_closing_date if training_closing_date else closing_day],
    "target": [""],
    "model_path": [""],
    "metrics_path": [""],
    "metrics_train": [""],
    "metrics_test": [""],
    "varimp": [""],
    "algorithm": [""],
    "author_login": ["csanc109"],
    "extra_info": [""],
    "scores_extra_info_headers": [";".join(EXTRA_INFO_COLS)],
    "year":  [year_partition],
    "month": [month_partition],
    "day":   [day_partition],
    "time": [int(executed_at.split(" ")[1].replace(":", ""))]
})

df_parameters = spark.createDataFrame(df_pandas).withColumn("day", col("day").cast("integer"))\
                                                .withColumn("month", col("month").cast("integer"))\
                                                .withColumn("year", col("year").cast("integer"))\
                                                .withColumn("time", col("time").cast("integer"))

'''
MODEL SCORES
'''

# set to null the columns that go to extra_info field and have no value
for col_ in EXTRA_INFO_COLS:
    df_model_scores = df_model_scores.withColumn(col_,
                                                 when(coalesce(length(col(col_)), lit(0)) == 0, MODEL_OUTPUTS_NULL_TAG).otherwise(
                                                     col(col_)))

df_model_scores = (
        df_model_scores.withColumn("extra_info", concat_ws(";", *[col(col_name) for col_name in EXTRA_INFO_COLS]))
        .withColumn("prediction", lit("0"))
        .drop(*EXTRA_INFO_COLS)
        .withColumnRenamed("comb_score", "scoring")
        .withColumn("model_name", lit("mobile_base"))
        .withColumn("executed_at", lit(executed_at))
        .withColumn("model_executed_at", lit(executed_at))
        .withColumn("year", lit(year_partition).cast("integer"))
        .withColumn("month",lit(month_partition).cast("integer"))
        .withColumn("day", lit(day_partition).cast("integer"))
        .withColumn("time", regexp_replace(split(col("executed_at"), " ")[1], ":", "").cast("integer"))
        .withColumn("predict_closing_date", lit("20190930"))
        .withColumn("model_output", lit(""))
    )

df_model_scores = ensure_types_model_scores_columns(df_model_scores, check_ids=False)



In [ ]:
day_path1 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=21/time=1574782575"
day_path2 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=22/time=1574791124"
day_path3 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=23/time=1574863576"
day_path4 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=24/time=1574791742"
day_path5 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=25/time=1574886360"
day_path6 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=26/time=1574887287"
day_path7 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=27/time=1574888086"
day_path8 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=28/time=1574944954"
day_path9 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=29/time=1574947667"
day_path10 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=30/time=1574941173"

paths_dict = {"20191021" : day_path1,
              "20191022" : day_path2,
              "20191023" : day_path3,
              "20191024" : day_path4,
              "20191025" : day_path5,
              "20191026" : day_path6,
              "20191027" : day_path7,
              "20191028" : day_path8,
              "20191029" : day_path9,
              "20191030" : day_path10
             }



#label_kind = "mobile"
label_kind = "mobile" 
closing_day = "20191023"

for closing_day in paths_dict.keys():

    df_labels = spark.read.load(paths_dict[closing_day])

    print("NUM LABELS ORIG", df_labels.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])

    if label_kind in  ["mobile+fix"]:
        df_lab = get_label(spark, closing_day, kind=label_kind, churn_window=15).select("msisdn", "label")
        df_labels = df_labels.drop("label")
        df_labels = df_labels.join(df_lab, on=["msisdn"], how="left").na.fill({'label': 0.0})

    elif label_kind != "mobile":
        print("Unknown kind {}. Program will exit here!".format(kind))
        import sys
        sys.exit()

    print("NUM LABELS AFTER", df_labels.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])


    from pyspark.sql.functions import avg as sql_avg
    myschema = df_labels.schema
    df_labels = df_labels.sort(desc("model_score"))
    df_top_6500 = spark.createDataFrame(df_labels.head(6500), schema=myschema)

    tt_churn_ref = df_top_6500.select(sql_avg('label').alias('churn_ref')).rdd.first()['churn_ref']
    print(closing_day, tt_churn_ref * 100)

# Incrementals

In [ ]:
from churn_nrt.src.utils.date_functions import days_range
   
from churn_nrt.src.utils.date_functions import move_date_n_days
from pyspark.sql.functions import avg as sql_avg



day_path1 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=21/time=1574782575"
day_path2 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=22/time=1574791124"
day_path3 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=23/time=1574863576"
day_path4 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=24/time=1574791742"
day_path5 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=25/time=1574886360"
day_path6 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=26/time=1574887287"
day_path7 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=27/time=1574888086"
day_path8 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=28/time=1574944954"
day_path9 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=29/time=1574947667"
day_path10 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=30/time=1574941173"
day_path11 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=31/time=1576059964"

#range_dates = days_range("20191021", "20191025")
range_dates = ["20191030"] 
print(range_dates)
paths_dict = {
               "20191021" : day_path1,
               "20191022" : day_path2,
               "20191023" : day_path3,
               "20191024" : day_path4,
               "20191025" : day_path5,
               "20191026" : day_path6,
               "20191027" : day_path7,
               "20191028" : day_path8,
               "20191029" : day_path9,
               "20191030" : day_path10,
               "20191031" : day_path11
            }

#label_kind = "mobile"
label_kind = "mobile+fix" 
top_ = 727
step_=1
verbose=True
compute_lift=True

for closing_day in range_dates:
    
    closing_day_B = move_date_n_days(closing_day, n=step_)
    print(" * - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - *")
    if closing_day in paths_dict.keys():
        print(closing_day, paths_dict[closing_day])
    else:
        print("no existe")
        continue
        
    if closing_day_B in paths_dict.keys():
        print(closing_day_B, paths_dict[closing_day_B])
    else:
        print("no existe")
        continue

    df_labels_A = spark.read.load(paths_dict[closing_day])
    df_labels_B = spark.read.load(paths_dict[closing_day_B])
    df_labels_join = df_labels_A.select("msisdn").join(df_labels_B.select("msisdn", "label", "model_score"), ['msisdn'], 'right').where(df_labels_A['msisdn'].isNull())
    df_labels_join = df_labels_join.cache()
    if verbose: print("volumen incremental", df_labels_join.count())

    #print("NUM LABELS ORIG", df_labels_join.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])

    if label_kind in  ["mobile+fix", "mobile"]:
        df_lab = get_label(spark, closing_day_B, kind=label_kind, churn_window=15).select("msisdn", "label")
        df_labels_join = df_labels_join.drop("label")
        df_labels_join = df_labels_join.join(df_lab, on=["msisdn"], how="left").na.fill({'label': 0.0})
        
    elif label_kind != "mobile":
        print("Unknown kind {}. Program will exit here!".format(label_kind))
        import sys
        sys.exit()

    #print("NUM LABELS AFTER", df_labels_join.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])

    tt_churn_ref = df_labels_join.select(sql_avg('label').alias('churn_ref')).rdd.first()['churn_ref']
    if verbose: print("churn_rate", tt_churn_ref)
    
    myschema3 = df_labels_join.schema
    
    if top_:
        from pyspark.sql.functions import avg as sql_avg
        df_labels_join = df_labels_join.sort(desc("model_score"))
        df_top_400 = spark.createDataFrame(df_labels_join.head(top_), schema=myschema3)
        tt_churn_ref = df_top_400.select(sql_avg('label').alias('churn_ref')).rdd.first()['churn_ref']
        print("------", label_kind, closing_day, closing_day_B, "ChRate{}".format(top_), tt_churn_ref * 100)
        
    if compute_lift:
        from churn_nrt.src.projects_utils.models.modeler import get_cumulative_lift_fix_step
        cum_churn_rate = get_cumulative_lift_fix_step(spark, df_labels_join, ord_col =  'model_score', label_col = 'label', step_=500)

# Referencia de Insights

In [ ]:
msisdn_df.unpersist()
msisdn_df = spark.read.table("raw_es.campaign_msisdncontacthist")
msisdn_df = msisdn_df.filter((col("CampaignCode").contains("COMP_WEB")) & (col("Creatividad").startswith("LLAM_"))).drop_duplicates(["msisdn"])
msisdn_df = msisdn_df.where(col("year")==2019).where(col("month")==10)
#msisdn_df = msisdn_df.persist()
msisdn_df.groupby("year", "month", "day").agg(sql_count("*").alias("count")).sort(desc("year"), desc("month"), desc("day")).show(100)

In [ ]:
closing_day = "20191027"


day_path1 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=21/time=1574782575"
day_path2 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=22/time=1574791124"
day_path3 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=23/time=1574863576"
day_path4 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=24/time=1574791742"
day_path5 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=25/time=1574886360"
day_path6 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=26/time=1574887287"
day_path7 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=27/time=1574888086"
day_path8 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=28/time=1574944954"
day_path9 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=29/time=1574947667"
day_path10 = "/data/udf/vf_es/churn/triggers/nav_comp_tests_all_labels/year=2019/month=10/day=30/time=1574941173"



paths_dict = {"20191021" : day_path1,
              "20191022" : day_path2,
              "20191023" : day_path3,
              "20191024" : day_path4,
              "20191025" : day_path5,
              "20191026" : day_path6,
              "20191027" : day_path7,
              "20191028" : day_path8,
              "20191029" : day_path9,
              "20191030" : day_path10
             }

df_car = spark.read.load(paths_dict[closing_day])

print("NUM LABELS original", df_car.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])


df_car = df_car.drop("label")
df_lab = get_label(spark, closing_day, kind="mobile", churn_window=15)

df_car_mobile = df_car.join(df_lab, on=["msisdn"], how="left").na.fill({'label': 0.0})

print("NUM LABELS mobile", df_car_mobile.select(sql_sum('label').alias('num_churners')).rdd.first()['num_churners'])



In [ ]:

def get_label(spark, closing_day, kind="mobile", churn_window=15, columns="simple"):
    from churn_nrt.src.utils.date_functions import move_date_n_days
    #start_port = move_date_n_days(closing_day, n=1)
    end_port = move_date_n_days(closing_day, n=churn_window)

    if kind=="mobile":

        from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests, get_customer_base
        df_target = get_mobile_portout_requests(spark, closing_day, end_port).select("msisdn", "label_mob").withColumnRenamed("label_mob", "label")
        return df_target
    
    elif kind=="mobile+fix":
        
        print("Computing target with mobile and fix sopo")
        

        from churn_nrt.src.data.sopos_dxs import FixPort
        from churn_nrt.src.data.customer_base import CustomerBase

        # The base of active services on closing_day
        from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests, get_customer_base
        base_df = get_customer_base(spark, closing_day).select('msisdn', "nif_cliente", "rgu")
        
        
        
        # Getting portout requests for fix and mobile services, and disconnections of fbb services
        print("******* Asking for FixPort...")
        df_sopo_fix = FixPort(spark).get_module(closing_day, save=False, churn_window=churn_window)
        df_sopo_fix = (df_sopo_fix.join(base_df.select("msisdn", "nif_cliente"), ['msisdn'], "left").na.fill({'label_srv': 0.0}))
        # Agg by nif_cliente the column label_srv just to be sure that only one per nif
        window_nc = Window.partitionBy("nif_cliente")
        df_sopo_fix = df_sopo_fix.withColumn('label_srv', sql_max('label_srv').over(window_nc)).na.fill({'label_srv': 0.0})     
        df_sopo_fix = df_sopo_fix.drop_duplicates(["nif_cliente"])


        base_df = base_df.join(get_mobile_portout_requests(spark, closing_day, end_port).select('msisdn', 'label_mob'), ['msisdn'], 'left').na.fill(0.0)
        df_sopos = (base_df.join(df_sopo_fix.select("nif_cliente", "label_srv"), ['nif_cliente'], "left").na.fill({'label_srv': 0.0}))

        df_sopos = df_sopos.withColumn('label', when((col('label_srv') == 1.0) | (col('label_mob') == 1.0), 1.0).otherwise(0.0))
        
        if columns=="simple":
            return df_sopos.filter(col('rgu') == 'mobile').select("msisdn", "label")
        else:
            return df_sopos.filter(col('rgu') == 'mobile').select("msisdn", "label", "label_mob", "label_srv")
    
    else:
        print("Unknown kind {}. Program will exit here!".format(kind))
        return None
                
        
#get_label(spark, closing_day="20191024", kind="mobile")

# Churn Rate - Insights campaign

In [ ]:

from churn_nrt.src.utils.date_functions import days_range
    
from churn_nrt.src.utils.date_functions import move_date_n_days
from pyspark.sql.functions import avg as sql_avg

#label_kind = "mobile"
#label_kind = "mobile+fix" 


# for closing_day in range_dates:
    
#     print(closing_day, paths_dict[closing_day])
  
#     msisdn_df = spark.read.table("raw_es.campaign_msisdncontacthist")

#     insights_df = msisdn_df.filter((col('year')==int(closing_day[:4])) & (col('month') == int(closing_day[4:6])) & (col('day') == int(closing_day[6:])) & (col("CampaignCode").contains("COMP_WEB")) & (col("Creatividad").startswith("LLAM_"))).select("msisdn").distinct()
#     print("clients", insights_df.count())
#     #start_port = move_date_n_days(closing_day, n=1)
#     end_port = move_date_n_days(closing_day, n=15)

#     from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests, get_customer_base
#     #insights_df.join(get_mobile_portout_requests(spark, start_port, end_port).select("msisdn", "label_mob").withColumnRenamed("label_mob", "label"), ['msisdn'], 'left').na.fill({'label': 0.0}).select(sql_avg('label')).show()
#     insights_df.join(get_label(spark, closing_day, kind=label_kind, churn_window=15).select("msisdn", "label"), ['msisdn'], 'left').na.fill({'label': 0.0}).select(sql_avg('label')).show()


base_closing_day = "20191031"
base_closing_day_end = "20191031"
port_closing_day = base_closing_day_end
label_kind = "mobile+fix" 

#msisdn_df.unpersist()
msisdn_df = (spark.read.table("raw_es.campaign_msisdncontacthist")
                .where( ((col("CampaignCode").contains("COMP_WEB")) & (col("Creatividad").startswith("LLAM_"))))
                .withColumn("datedate", concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0'))).cache()
                .where(   (col('datedate')>= base_closing_day) & (col('datedate')<= base_closing_day_end))
                .drop_duplicates(["msisdn"]))
print("clients", msisdn_df.count())
print("drop_duplic", msisdn_df.drop_duplicates(["msisdn"]).count())
msisdn_df.groupby("year", "month", "day").agg(sql_count("*").alias("count")).sort(desc("year"), desc("month"), desc("day")).show(100)
#start_port = move_date_n_days(closing_day, n=1)
#end_port = move_date_n_days(base_closing_day_end, n=15)
msisdn_df.groupby("datedate").agg(sql_count("*").alias("count")).sort(desc("datedate")).show(100)

from churn.analysis.triggers.base_utils.base_utils import get_mobile_portout_requests, get_customer_base
#insights_df.join(get_mobile_portout_requests(spark, start_port, end_port).select("msisdn", "label_mob").withColumnRenamed("label_mob", "label"), ['msisdn'], 'left').na.fill({'label': 0.0}).select(sql_avg('label')).show()
msisdn_df.join(get_label(spark, base_closing_day_end, kind=label_kind, churn_window=15).select("msisdn", "label"), ['msisdn'], 'left').na.fill({'label': 0.0}).select(sql_avg('label')).show()

    

In [ ]:
msisdn_df = spark.read.table("raw_es.campaign_msisdncontacthist")
msisdn_df = msisdn_df.filter((col("CampaignCode").contains("COMP_WEB")) & (col("Creatividad").startswith("LLAM_"))).drop_duplicates(["msisdn"])
msisdn_df = msisdn_df.where(col("year")==2019).where(col("month")>=10)

msisdn_df.groupby("year", "month", "day").agg(sql_count("*").alias("count")).sort(desc("year"), desc("month"), desc("day")).show(100)

In [ ]:
df_bd = spark.read.option("delimeter", "|").option("header",True).csv("/user/csanc109/data/mobile_base_20190930_desanonim/Mobile_Base-20190930.csv")
df_bd.count()

In [65]:
spark.read.load("/data/raw/vf_es/customerprofilecar/ADOBE_SECTIONS/1.1/parquet").count()

AnalysisException: u'Path does not exist: hdfs://nameservice1/data/raw/vf_es/customerprofilecar/ADOBE_SECTIONS/1.1/parquet;'

# ADOBE DATA

In [123]:
df_net = spark.read.table("raw_es.customerprofilecar_adobe_sections").drop("service_processed_at", "service_file_id", "year", "month", "day").withColumn("msisdn", col("MSISDN").substr(3, 9))
df_net.show(truncate=False)

+-------------------+---------+----------------------------------------------------------------------------+
|Date               |msisdn   |Pages                                                                       |
+-------------------+---------+----------------------------------------------------------------------------+
|2019-11-19 00:08:36|B42000417|productos y servicios:resumen de productos y servicios                      |
|2019-11-19 00:08:45|B42000417|productos y servicios:detalle de tarifa:vodafone tv online                  |
|2019-11-19 00:16:33|B42000417|productos y servicios:resumen de productos y servicios                      |
|2019-11-19 00:16:41|B42000417|productos y servicios:detalle de tarifa:vodafone tv online                  |
|2019-11-19 00:17:17|B42000417|productos y servicios:detalle de tarifa:vodafone tv online:cambio de usuario|
|2019-11-19 00:17:57|B42000417|dashboard:home                                                              |
|2019-11-19 00:21:1

In [126]:
#df_net.unpersist()



import datetime as dt



df_net = spark.read.table("raw_es.customerprofilecar_adobe_sections").drop("service_processed_at", "service_file_id", "year", "month", "day").withColumn("msisdn", col("MSISDN").substr(3, 9))

func_date =  udf (lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime("%Y%m%d"), StringType())

df_net = df_net.withColumn('date_', func_date(col('Date'))).drop("Date")

stages_name = ["section", "subsection", "page_name"]

for ii, col_ in enumerate(stages_name):
    if col_ == "section":
        # Do not split if Pages is an url
        df_net = df_net.withColumn(col_, when(~col("Pages").rlike("^https://"), split("Pages", ":")[ii]).otherwise(col("Pages")))
    else:
        df_net = df_net.withColumn(col_, when(~col("Pages").rlike("^https://"), split("Pages", ":")[ii]).otherwise(None))

df_net = df_net.withColumn('section', regexp_replace('section', ' ', '_'))  # remove spaces with underscore to avoid error when writing df
df_net = df_net.withColumn("section", when(col("section").rlike("https://m.vodafone.es/mves/Dashboard/Ayuda"), "url_ayuda").otherwise(col("section")))
#df_net.groupby("section").agg(sql_count("*").alias("count")).sort(desc("count")).show(truncate=False)

#print("[MyVFdata] Filtering sections other than {}".format(",".join(sections)))
#df_net = df_net.where(col("section").isin(sections))
df_net.groupby("section").agg(sql_count("*").alias("count")).sort(desc("count")).show(truncate=False)

df_net = df_net.withColumn("days_since_nav", when(col("date_").isNotNull(),
                                                  datediff(from_unixtime(unix_timestamp(lit(closing_day), "yyyyMMdd")), from_unixtime(unix_timestamp(col("date_"), "yyyyMMdd")))).otherwise(-1))
df_net = df_net.withColumn("diff", when(col("days_since_nav") != -1,
                                        lag(col("days_since_nav"), -1).over(Window.partitionBy("msisdn").orderBy(asc("days_since_nav"))) - col("days_since_nav")).otherwise(None))



df_net = df_net.withColumn('date_', func_date(col('Date')))

#df_net.show(truncate=False)



+---------------------+--------+
|section              |count   |
+---------------------+--------+
|dashboard            |17580672|
|facturas             |5390241 |
|productos_y_servicios|5381573 |
|que_tengo_contratado |3607038 |
|mi_cuenta            |2792485 |
|mensajes             |2054267 |
|prelogin             |1573449 |
|childbrowser         |1012288 |
|webview              |978901  |
|consumo              |927399  |
|screen_myrewards     |816450  |
|averias              |737563  |
|overlay_promo_bf     |366341  |
|faqs                 |354766  |
|login                |324253  |
|ajustes_de_linea     |314110  |
|need_help            |309608  |
|ajustes              |295724  |
|screen_allrewards    |279236  |
|screen_rewarddetails |277214  |
+---------------------+--------+
only showing top 20 rows



NameError: name 'from_unixtime' is not defined

In [ ]:
#sections = df_net.select("section").distinct().rdd.map(lambda x: x[0]).collect()
df_net.select("section", "subsection", "page_name").groupby("section", "subsection", "page_name").agg(sql_count("*").alias("count")).sort(desc("count")).show(100, truncate=False)

In [ ]:
#sections = df_net.select("section").distinct().rdd.map(lambda x: x[0]).collect()
sections = [u'https://m.vodafone.es/mves/billing/currentConsumption',
 u'lo_sentimos,_el_servicio_no_esta_disponible',
 u'television',
 u'https://m.vodafone.es/mves/myAccount/permissions',
 u'https://m.vodafone.es/mves/myAccount/billingAccounts',
 u'https://m.vodafone.es/mves/TopUp/topUpHistory',
 u'settings_privacy_page',
 u'https://m.vodafone.es/mves/splash',
 u'service_settings',
 u'https://m.vodafone.es/mves/billing/eBill',
 u'roaming_warning_popup',
 u'https://m.vodafone.es/mves/ProductsAndServices/securenet',
 u'overlay_informacion_5g',
 u'mis_dispositivos',
 u'https://m.vodafone.es/mves/PurchasedProducts/couteDetails',
 u'https://m.vodafone.es/mves/Dashboard',
 u'childbrowser',
 u'ayuda_y_soporte',
 u'https://m.vodafone.es/mves/myAccount/nif',
 u'https://m.vodafone.es/mves/billing/billingOverview',
 u'https://m.vodafone.es/mves/LineServices/CallOptions/DictaSMS',
 u'https://m.vodafone.es/mves/ServiceSettings/thirdPartyChild',
 u'https://m.vodafone.es/mves/myAccount/details',
 u'https://m.vodafone.es/mves/myAccount/PermissionsAndPreferences/DiagnosticoDeMiVodafone',
 u'network_usage_error',
 u'dictasms',
 u'avisos_de_llamada_perdida',
 u'MyVodafone_Production/2098',
 u'Speed_Checker_Error',
 u'ajustes_y_extras',
 u'https://m.vodafone.es/mves/billing/creditNotes',
 u'https://m.vodafone.es/mves/myAccount/serviceSelectorCommitmentContract',
 u'que_tengo_contratado',
 u'overlay_maintenance',
 u'network_usage_reset_confirmation_page',
 u'MyVodafone_Production/1917',
 u'https://m.vodafone.es/mves/ProductsAndServices/extrasList',
 u'Running_Speed_Test',
 u'https://m.vodafone.es/mves/myAccount/commitmentContracts',
 u'https://m.vodafone.es/mves/FaultManagement/RecentTickets',
 u'https://m.vodafone.es/mves/myAccount/editDetails',
 u'asistente',
 u'inicio',
 u'https://m.vodafone.es/mves/ProductsAndServices/MainScreen',
 u'facturas',
 u'Speed_Checker_Landing_Page',
 u'upgrade',
 u'https://m.vodafone.es/mves',
 u'https://m.vodafone.es/mves/login/companyChooser',
 u'Permission_Request',
 u'https://m.vodafone.es/mves/ProductsAndServices/PlanDetails',
 u'login',
 u'netperform_privacy_policy',
 u'https://m.vodafone.es/mves/ProductsAndServices/details',
 u'overlay_favourite',
 u'https://m.vodafone.es/mves/ProductsAndServices/ManageMiWifi/MiWifiLanding',
 u'https://m.vodafone.es/mves/myAccount/PermissionsAndAuthorizations',
 u'screen_allrewards',
 u'https://m.vodafone.es/mves/ServiceSettings/purchasesAndSubscriptions',
 u'https://m.vodafone.es/mves/billing/billConfig',
 u'overlay_no_disponibilidad',
 u'Mi_Vodafone_4.4.7_(176)',
 u'https://m.vodafone.es/mves/myAccount/Mycontracts/MainPage',
 u'prelogin',
 u'screen_rewarddetails',
 u'faq',
 u'network_usage_landing_page',
 u'https://m.vodafone.es/mves/myAccount/OverviewScreen',
 u'https://m.vodafone.es/mves/login',
 u'tarifa',
 u'https://m.vodafone.es/mves/billing/BillOverview',
 u'https://m.vodafone.es/mves/ProductsAndServices/superWifiDetails',
 u'migracion_destiny',
 u'faqs',
 u'https://m.vodafone.es/mves/billing/billingDetails',
 u'https://m.vodafone.es/mves/ProductsAndServices/update',
 u'permission_gps_request',
 u'https://m.vodafone.es/mves/myAccount/PermissionsAndPreferences',
 u'https://m.vodafone.es/mves/licenses',
 u'https://m.vodafone.es/mves/myAccount/Serviceaccess/MainPage',
 u'dashboard',
 u'https://m.vodafone.es/mves/PurchasedProducts/MainScreen',
 u'Netperform_Privacy_policy',
 u'purchase',
 u'url_ayuda',
 u'https://m.vodafone.es/mves/LineServices/CallOptions/SMSAlert',
 u'productsandservices',
 u'https://m.vodafone.es/mves/impersonate',
 u'https://m.vodafone.es/mves/dashboard',
 u'https://m.vodafone.es/mves/Promotions/xmas',
 u'https://m.vodafone.es/mves/myAccount/editPassword',
 u'https://m.vodafone.es/mves/login/loginSeibelRedirection',
 u'https://m.vodafone.es/mves/TopUp/currentSpendingHistory',
 u'MyVodafone_Production/1767',
 u'gestionar_identificador_llamada',
 u'ctc_sin_tarifa',
 u'promo_navidad',
 u'https://m.vodafone.es/mves/FaultManagement/LandingPage',
 u'MyVodafone_Production/2052',
 u'necesitas_mas_ayuda',
 u'https://m.vodafone.es/mves/serviceSettings/pinpuk',
 u'mi_cuenta',
 u'averias',
 u'\xbfNecesitas_m\xe1s_ayuda?',
 u'push_notification',
 u'https://m.vodafone.es/mves/FaultManagement/Categories',
 u'https://m.vodafone.es/mves/myAccount/prefrences',
 u'https://m.vodafone.es/mves/ecareLanding',
 u'https://m.vodafone.es/mves/billing/CurrentSpendingConsumptionDetails',
 u'https://m.vodafone.es/mves/LineServices/CallOptions/Voicemail',
 u'https://m.vodafone.es/mves/billing/deviceInvoice',
 u'https://m.vodafone.es/mves/billing/billPaymentDetails',
 u'https://m.vodafone.es/mves/myAccount/accessInfo',
 u'https://m.vodafone.es/mves/ProductsAndServices/newChannelList',
 u'bienvenido_a_destiny',
 u'red',
 u'factura',
 u'internet,_fijo_y_vodafone_one',
 u'running_speed_test',
 u'https://m.vodafone.es/mves/FaultManagement/OpenNewTicket',
 u'https://m.vodafone.es/mves/TopUp/topUpPrepaidservice',
 u'https://m.vodafone.es/mves/login/orderMangment',
 u'https://m.vodafone.es/mves/contingency',
 u'https://m.vodafone.es/mves/serviceSettings/PINPUKManagement',
 u'ajustes',
 u'https://m.vodafone.es/mves/myAccount/YourDetails/MainPage',
 u'https://m.vodafone.es/mves/myAccount/permissionAndPrefrence',
 u'promo_bf',
 u'aviso_legal',
 u'overlay_promo_bf',
 u'overlay_discover',
 u'https://m.vodafone.es/mves/billing/billPayment',
 u'yetalive',
 u'overlay_offline',
 u'permisos',
 u'screen_myrewards',
 u'https://m.vodafone.es/mves/serviceSettings/thirdPartyLanding',
 u'https://m.vodafone.es/mves/myAccount/contracts',
 u'previous_speed_test_results',
 u'aviso_registro_mi_vodafone.',
 u'productos_y_servicios',
 u'https://m.vodafone.es/mves/billing/Abonos',
 u'webview',
 u'https://m.vodafone.es/mves/recommender',
 u'https://m.vodafone.es/mves/billing/BillDetails/DevicesInvoices',
 u'vodafone',
 u'avisos_para_rellamadas',
 u'https://m.vodafone.es/mves/LineServices/Restrictions/MainPage',
 u'promotions',
 u'speed_checker_error',
 u'https://m.vodafone.es/mves/myAccount/Paymethods/MainPage',
 u'bienvenido',
 u'miercoyes',
 u'overlay_no_codes_left',
 u'need_help',
 u'mi_vodafone',
 u'popup_confirmacion_permisos',
 u'https://m.vodafone.es/mves/Promotions/onboarding',
 u'Speed_Test_Results',
 u'aviso',
 u'llamadas_en_espera',
 u'https://m.vodafone.es/mves/nuevosBeneficios',
 u'condiciones_de_uso_del_logado_por_red',
 u'ocultar_tu_numero',
 u'https://m.vodafone.es/mves/billing/BillConfigurations',
 u'registro',
 u'https://m.vodafone.es/mves/ServiceSettings/PINPUKManagement',
 u'https://m.vodafone.es/mves/ServiceSettings/RestrictionsThirdPartyLandingPage',
 u'avisos',
 u'screen_raffles',
 u'https://m.vodafone.es/mves/Inbox/InboxListing',
 u'app_launch',
 u'https://m.vodafone.es/mves/LineServices/CallOptions/Waiting',
 u'destiny_migracion_septiembre',
 u'https://m.vodafone.es/mves/myAccount/gdprConfirmPermission',
 u'gracias_por_utilizar_mi_vodafone',
 u'mensajes',
 u'home',
 u'opciones_de_llamada',
 u'ventajas_exclusivas',
 u'Asistente',
 u'permission_request',
 u'https://m.vodafone.es/mves/ProductsAndServices/Extras/Entertainment/Listing',
 u'igracion_destiny',
 u'https://m.vodafone.es/mves/billing/BillDetails',
 u'',
 u'https://m.vodafone.es/mves/Inbox/details',
 u'MyVodafone_Production/1822',
 u'consumo',
 u'datos_personales',
 u'registrop',
 u'ajustes_de_linea',
 u'https://m.vodafone.es/mves/myAccount/editEmail',
 u'https://m.vodafone.es/mves/myAccount/otp',
 u'detalle_de_tarifas',
 u'https://m.vodafone.es/mves/myAccount/siteSelector',
 u'FAQs',
 u'https://m.vodafone.es/mves/ProductsAndServices/combi',
 u'https://m.vodafone.es/mves/LineServices/MobileConnect',
 u'https://m.vodafone.es/mves/LineServices/CallOptions/HidemyNumber',
 u'recargas',
 u'Previous_Speed_Test_Results',
 u'https://m.vodafone.es/mves/ProductsAndServices/Prepaidenjoymore/Listing',
 u'null',
 u'speed_test_results',
 u'screen_redeemedrewards',
 u'contigencia',
 u'https://m.vodafone.es/mves/billing/billingDetailsServiceUsages',
 u'https://m.vodafone.es/mves/LineServices/CallOptions/MainPage',
 u'https://m.vodafone.es/mves/LineServices/CallOptions/Autoredial',
 u'https://m.vodafone.es/mves/LineServices/VbyVodafone',
 u'MyVodafone_Production/1970',
 u'gestion_de_lineas',
 u'https://m.vodafone.es/mves/billing/noBills']

In [ ]:
df_net.show(truncate=False)

In [ ]:
df_customer = spark.read.load("/data/udf/vf_es/amdocs_ids/customer/year=2019/month=9/day=30")
df_customer.where(col("num_cliente")=='462624393').show()

In [ ]:
df_service = spark.read.load("/data/udf/vf_es/amdocs_ids/service/year=2019/month=9/day=30")

df_service.where(col("HOMEZONE").isNotNull()).select("SRV_BASIC", "RGU", "HOMEZONE", "DESC_SRV_BASIC").distinct().show(100, truncate=False)
# +---------+------+--------+
# |SRV_BASIC|RGU   |HOMEZONE|
# +---------+------+--------+
# |TRDNP    |fixed |VFETC   |
# |TESTA    |fixed |VFETC   |
# |VFETC    |fixed |VFETC   |
# |MRPD1    |mobile|VFETC   |
# |TESTH    |mobile|VFETC   |
# |TESTH    |fixed |VFETC   |
# +---------+------+--------+




In [ ]:
df_service.where(col("HOMEZONE").isNotNull()).select("SRV_BASIC", "RGU", "HOMEZONE", "DESC_SRV_BASIC").groupby(col("SRV_BASIC"), col("HOMEZONE"), col("RGU")).agg(sql_count("*")).show()


In [ ]:
df_service_new.columns

In [ ]:
df_service_new = spark.read.load("/data/udf/vf_es/amdocs_inf_dataset/service/year=2019/month=9/day=21")

df_service_new.where( col("Serv_HOMEZONE").isNotNull() & (col("Serv_HOMEZONE") != "unknown")).select("Serv_SRV_BASIC", "Serv_RGU", "Serv_HOMEZONE", "Serv_DESC_SRV_BASIC").groupby(col("Serv_SRV_BASIC"), col("Serv_HOMEZONE"), col("Serv_RGU")).agg(sql_count("*")).show()


In [111]:
closing_day = "20190930"
import datetime as dt
from pyspark.sql.functions import (col, lit, lower, concat, count, max, avg, desc, asc, row_number, lpad, trim, when, split, to_date, coalesce,  isnull)

def is_null_date(fecha):
    return year(fecha) == 1753

from pyspark.sql.functions import  (sum as sql_sum, countDistinct, trim
                                    ,max, greatest , split,desc, col, current_date
                                    , datediff, lit, translate, udf
                                    , when, concat_ws, concat, decode, length
                                    , substring, to_date, regexp_replace, lpad
                                    , hour, date_format, row_number
                                    , expr, coalesce, udf, year)
from pyspark.sql.window import Window


print('Getting Services Information...')
data_mapper = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',
                                                                    delimiter='|').load('/tmp/rbuendi1/blow/')
data_service_ori = (spark.read.load("/data/raw/vf_es/customerprofilecar/SERVICESOW/1.1/parquet/")
    # data_service_ori = (spark.read.table("raw_es.customerprofilecar_servicesow")
                    .where(
    concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day))

data_serviceprice_ori = (spark.read.load("/data/raw/vf_es/priceplanstariffs/PLANPRIC_ONO/1.0/parquet/")
    # data_serviceprice_ori = (spark.read.table("raw_es.planpric_ono")
                         .where(
    concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day))
w_srv = Window().partitionBy("OBJID").orderBy(desc("year"), desc("month"), desc("day"))
data_service_ori_norm = (data_service_ori
                         .withColumn("rowNum", row_number().over(w_srv))
                         .where(col('rowNum') == 1)
                         )

w_srv_price = Window().partitionBy("OBJID").orderBy(desc("year"), desc("month"), desc("day"))
data_serviceprice_ori_norm = (data_serviceprice_ori
                              .withColumn("rowNum", row_number().over(w_srv_price))
                              .withColumnRenamed('OBJID', 'OBJID2PRICE')
                              .withColumnRenamed('PRICE', 'PRICE2PRICE')
                              .where(col('rowNum') == 1)
                              )
# Params Load - In a not so far future it should be loaded from BDP MAESTRO_SERVICIOS
LOC_RT_PATH = '/data/udf/vf_es/ref_tables/amdocs_ids/'
LOC_RT_EXPORT_MAP = LOC_RT_PATH + 'RBL_EXPORT_MAP.TXT'
LOC_RT_PARAM_OW_SERVICES= LOC_RT_PATH + 'PARAM_OW_SERVICES.TXT'
data_service_param = spark.read.csv(LOC_RT_PARAM_OW_SERVICES, header=True,sep='\t')
#data_service_param = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter='\t').load('/tmp/rbuendi1/PARAM_AMD_DLAB/')
data_service_param = (data_service_param.where(col('rgu').isNotNull())
  .withColumn('rgu', 
    when(data_service_param['rgu'] == 'bam-movil', 'bam_mobile')
    .when(data_service_param['rgu'] == 'movil', 'mobile')
    .otherwise(data_service_param['rgu'])))

ClosingDay_date = dt.date(int(closing_day[:4]), int(closing_day[4:6]), int(closing_day[6:8]))
yesterday = ClosingDay_date + dt.timedelta(days=-1)

data_service_tmp1_basic = (data_service_ori_norm[
                               'OBJID', 'NUM_CLIENTE', 'NUM_SERIE', 'COD_SERVICIO', 'ESTADO_SERVICIO', 'FECHA_INST', 'FECHA_CAMB', 'INSTANCIA', 'PRIM_RATE', 'SERVICIOS_ACTUALES2PRICE', 'year', 'month', 'day']
                           # .where(col('NUM_CLIENTE')=='758012325')   # aplicamos filtro para muestreo
                           .where(((to_date(col('FECHA_CAMB')) >= yesterday)) | (col('FECHA_CAMB').isNull()) | (is_null_date(col('FECHA_CAMB'))))
                           .withColumn("Instancia_P", trim(split(data_service_ori_norm.INSTANCIA, '\\.')[0]))
                           .withColumn("Instancia_S", split(data_service_ori_norm.INSTANCIA, '\\.')[1])
                           .withColumn("TACADA", concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')))
                           .join(data_service_param['COD_SERVICIO', 'DESC_SERVICIO', 'RGU', 'RGU_mobile', 'RGU_BAM', 'TIPO', 'LEGACY'],["COD_SERVICIO"], 'inner')
                           .join(data_serviceprice_ori_norm['OBJID2PRICE', 'PRICE2PRICE'],
                                 col('SERVICIOS_ACTUALES2PRICE') == col('OBJID2PRICE'), 'leftouter')
                           .withColumn('MSISDN',
                                       when((col('Instancia_S').isNull() & (col('COD_SERVICIO') == 'TVOTG')),
                                            concat(lit('FICT_TVOTG_'), data_service_ori_norm.NUM_CLIENTE))
                                       .when((col('Instancia_S').isNull() & (col('COD_SERVICIO') != 'TVOTG')),
                                             data_service_ori_norm.NUM_SERIE)
                                       .otherwise(lit(None)))
                           .withColumn('SERV_PRICE',
                                       when((col('PRIM_RATE').isNull()) | (trim(col('PRIM_RATE')) == ''),
                                            data_serviceprice_ori_norm.PRICE2PRICE.cast('double'))
                                       .otherwise(data_service_ori_norm.PRIM_RATE.cast('double'))
                                       )
                           )
data_service_tmp2_basic = (data_service_tmp1_basic
        .groupBy('NUM_CLIENTE', 'Instancia_P')
        .agg(
        max(col('MSISDN')).alias("MSISDN")
        , max(when(col('Instancia_S').isNull(), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("SRV_BASIC")
        , max(when(col('Instancia_S').isNull(), data_service_tmp1_basic.DESC_SERVICIO)
              .otherwise(None)).alias("DESC_SRV_BASIC")
        , max(when(col('Instancia_S').isNull(), data_service_tmp1_basic.OBJID)
              .otherwise(None)).alias("OBJID")
        , max(when(col('Instancia_S').isNull(), data_service_tmp1_basic.TACADA)
              .otherwise(None)).alias("TACADA")
        , max(when(col('Instancia_S').isNull(), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_SRV_BASIC")
        , max(when(col('Instancia_S').isNull(), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_SRV_BASIC")
        , max(when(col('Instancia_S').isNull(), data_service_tmp1_basic.RGU)
              .otherwise(None)).alias("RGU")
        , max(when((col('TIPO') == 'SIM'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TIPO_SIM")
        , max(when((col('TIPO') == 'SIM'), data_service_tmp1_basic.NUM_SERIE)
              .otherwise(None)).alias("IMSI")
        , max(when((col('TIPO') == 'TARIFA'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TARIFF")
        , max(when((col('TIPO') == 'TARIFA'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TARIFF")
        , max(when((col('TIPO') == 'TARIFA'), data_service_tmp1_basic.DESC_SERVICIO)
              .otherwise(None)).alias("DESC_TARIFF")
        , max(when((col('TIPO') == 'TARIFA'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TARIFF")
        , max(when((col('TIPO') == 'TARIFA_VOZ'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("VOICE_TARIFF")
        , max(when((col('TIPO') == 'TARIFA_VOZ'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_VOICE_TARIFF")
        , max(when((col('TIPO') == 'TARIFA_VOZ'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_VOICE_TARIFF")
        , max(when((col('TIPO') == 'MODULO_DATOS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("DATA")
        , max(when((col('TIPO') == 'MODULO_DATOS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_DATA")
        , max(when((col('TIPO') == 'MODULO_DATOS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_DATA")
        , max(when((col('TIPO').isin('DTO_NIVEL1')), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("DTO_LEV1")
        , max(when((col('TIPO').isin('DTO_NIVEL1')), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_DTO_LEV1")
        , max(when((col('TIPO') == 'DTO_NIVEL1'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_DTO_LEV1")
        , max(when((col('TIPO').isin('DTO_NIVEL2')), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("DTO_LEV2")
        , max(when((col('TIPO').isin('DTO_NIVEL2')), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_DTO_LEV2")
        , max(when((col('TIPO') == 'DTO_NIVEL2'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_DTO_LEV2")
        , max(when((col('TIPO').isin('DTO_NIVEL3')), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("DTO_LEV3")
        , max(when((col('TIPO').isin('DTO_NIVEL3')), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_DTO_LEV3")
        , max(when((col('TIPO') == 'DTO_NIVEL3'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_DTO_LEV3")
        , max(when((col('TIPO').isin('DATOS_ADICIONALES')), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("DATA_ADDITIONAL")
        , max(when((col('TIPO').isin('DATOS_ADICIONALES')), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_DATA_ADDITIONAL")
        , max(when((col('TIPO') == 'DATOS_ADICIONALES'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_DATA_ADDITIONAL")
        , max(when((col('TIPO') == 'OOB'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("OOB")
        , max(when((col('TIPO') == 'OOB'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_OOB")
        , max(when((col('TIPO') == 'OOB'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_OOB")
        , max(when((col('TIPO') == 'NETFLIX_NAPSTER'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("NETFLIX_NAPSTER")
        , max(when((col('TIPO') == 'NETFLIX_NAPSTER'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_NETFLIX_NAPSTER")
        , max(when((col('TIPO') == 'NETFLIX_NAPSTER'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_NETFLIX_NAPSTER")
        , max(when((col('TIPO') == 'ROAMING_BASICO'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("ROAMING_BASIC")
        , max(when((col('TIPO') == 'ROAMING_BASICO'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_ROAMING_BASIC")
        , max(when((col('TIPO') == 'ROAMING_BASICO'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_ROAMING_BASIC")
        , max(when((col('TIPO') == 'ROAM_USA_EUR'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("ROAM_USA_EUR")
        , max(when((col('TIPO') == 'ROAM_USA_EUR'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_ROAM_USA_EUR")
        , max(when((col('TIPO') == 'ROAM_USA_EUR'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_ROAM_USA_EUR")
        , max(when((col('TIPO') == 'ROAM_ZONA_2'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("ROAM_ZONA_2")
        , max(when((col('TIPO') == 'ROAM_ZONA_2'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_ROAM_ZONA_2")
        , max(when((col('TIPO') == 'ROAM_ZONA_2'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_ROAM_ZONA_2")
        , max(when((col('TIPO') == 'CONSUMO_MIN'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("CONSUM_MIN")
        , max(when((col('TIPO') == 'CONSUMO_MIN'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_CONSUM_MIN")
        , max(when((col('TIPO') == 'CONSUMO_MIN'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_CONSUM_MIN")
        , max(when(col('COD_SERVICIO') == 'SIMVF', 1)
              .otherwise(0)).alias("SIM_VF")
        , max(when((col('TIPO') == 'HOMEZONE'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("HOMEZONE")
        , max(when((col('TIPO') == 'HOMEZONE'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_HOMEZONE")
        , max(when((col('TIPO') == 'HOMEZONE'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_HOMEZONE")
        , max(when((col('TIPO') == 'HOMEZONE'), data_service_tmp1_basic.NUM_SERIE)
              .otherwise(None)).alias("MOBILE_HOMEZONE")
        , max(when((col('TIPO') == 'UPGRADE'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("FBB_UPGRADE")
        , max(when((col('TIPO') == 'UPGRADE'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_FBB_UPGRADE")
        , max(when((col('TIPO') == 'UPGRADE'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_FBB_UPGRADE")
        , max(when((col('TIPO') == 'DECO'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("DECO_TV")
        , max(when((col('TIPO') == 'DECO'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_DECO_TV")
        , max(when((col('TIPO') == 'DECO'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_DECO_TV")
        , max(when((col('TIPO') == 'DECO'), data_service_tmp1_basic.NUM_SERIE)
              .otherwise(None)).alias("NUM_SERIE_DECO_TV")
        , max(when((col('TIPO') == 'DECO'), data_service_tmp1_basic.OBJID)
              .otherwise(None)).alias("OBJID_DECO_TV")
        , max(when((col('TIPO') == 'TVCUOTAALTA'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_CUOTA_ALTA")
        , max(when((col('TIPO') == 'TVCUOTAALTA'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_CUOTA_ALTA")
        , max(when((col('TIPO') == 'TVCUOTAALTA'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_CUOTA_ALTA")
        , max(when((col('TIPO') == 'TV_PLANES_TARIFAS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_TARIFF")
        , max(when((col('TIPO') == 'TV_PLANES_TARIFAS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_TARIFF")
        , max(when((col('TIPO') == 'TV_PLANES_TARIFAS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_TARIFF")
        , max(when((col('TIPO') == 'TV_CUOTASCARGOS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_CUOT_CHARGES")
        , max(when((col('TIPO') == 'TV_CUOTASCARGOS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_CUOT_CHARGES")
        , max(when((col('TIPO') == 'TV_CUOTASCARGOS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_CUOT_CHARGES")
        , max(when((col('TIPO') == 'TVPROMOS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_PROMO")
        , max(when((col('TIPO') == 'TVPROMOS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_PROMO")
        , max(when((col('TIPO') == 'TVPROMOS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_PROMO")
        , max(when((col('TIPO') == 'TVPROMOUSER'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_PROMO_USER")
        , max(when((col('TIPO') == 'TVPROMOUSER'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_PROMO_USER")
        , max(when((col('TIPO') == 'TVPROMOUSER'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_PROMO_USER")
        , max(when((col('TIPO') == 'TV_ABONOS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_ABONOS")
        , max(when((col('TIPO') == 'TV_ABONOS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_ABONOS")
        , max(when((col('TIPO') == 'TV_ABONOS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_ABONOS")
        , max(when((col('TIPO') == 'TV_FIDELIZA'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_LOYALTY")
        , max(when((col('TIPO') == 'TV_FIDELIZA'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_LOYALTY")
        , max(when((col('TIPO') == 'TV_FIDELIZA'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_LOYALTY")
        , max(when((col('TIPO') == 'TV_SVA'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TV_SVA")
        , max(when((col('TIPO') == 'TV_SVA'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TV_SVA")
        , max(when((col('TIPO') == 'TV_SVA'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TV_SVA")
        , max(when((col('TIPO') == 'C_PLUS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("FOOTBALL_TV")
        , max(when((col('TIPO') == 'C_PLUS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_FOOTBALL_TV")
        , max(when((col('TIPO') == 'C_PLUS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_FOOTBALL_TV")
        , max(when((col('TIPO') == 'MOTOR'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("MOTOR_TV")
        , max(when((col('TIPO') == 'MOTOR'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_MOTOR_TV")
        , max(when((col('TIPO') == 'MOTOR'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_MOTOR_TV")
        , max(when((col('TIPO') == 'PVR'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("PVR_TV")
        , max(when((col('TIPO') == 'PVR'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_PVR_TV")
        , max(when((col('TIPO') == 'PVR'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_PVR_TV")
        , max(when((col('TIPO') == 'ZAPPER'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("ZAPPER_TV")
        , max(when((col('TIPO') == 'ZAPPER'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_ZAPPER_TV")
        , max(when((col('TIPO') == 'ZAPPER'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_ZAPPER_TV")
        , max(when((col('TIPO') == 'TRYBUY'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TRYBUY_TV")
        , max(when((col('TIPO') == 'TRYBUY'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TRYBUY_TV")
        , max(when((col('TIPO') == 'TRYBUY'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TRYBUY_TV")
        , max(when((col('TIPO') == 'TRYBUY_AUTOM'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TRYBUY_AUTOM_TV")
        , max(when((col('TIPO') == 'TRYBUY_AUTOM'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TRYBUY_AUTOM_TV")
        , max(when((col('TIPO') == 'TRYBUY_AUTOM'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TRYBUY_AUTOM_TV")

        , max(when((col('TIPO') == 'CINE'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("CINE")
        , max(when((col('TIPO') == 'CINE'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_CINE")
        , max(when((col('TIPO') == 'CINE'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_CINE")

        , max(when((col('TIPO') == 'SERIES'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("SERIES")
        , max(when((col('TIPO') == 'SERIES'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_SERIES")
        , max(when((col('TIPO') == 'SERIES'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_SERIES")

        , max(when((col('TIPO') == 'SERIEFANS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("SERIEFANS")
        , max(when((col('TIPO') == 'SERIEFANS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_SERIEFANS")
        , max(when((col('TIPO') == 'SERIEFANS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_SERIEFANS")

        , max(when((col('TIPO') == 'SERIELOVERS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("SERIELOVERS")
        , max(when((col('TIPO') == 'SERIELOVERS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_SERIELOVERS")
        , max(when((col('TIPO') == 'SERIELOVERS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_SERIELOVERS")

        , max(when((col('TIPO') == 'CINEFANS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("CINEFANS")
        , max(when((col('TIPO') == 'CINEFANS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_CINEFANS")
        , max(when((col('TIPO') == 'CINEFANS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_CINEFANS")

        , max(when((col('TIPO') == 'PEQUES'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("PEQUES")
        , max(when((col('TIPO') == 'PEQUES'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_PEQUES")
        , max(when((col('TIPO') == 'PEQUES'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_PEQUES")

        , max(when((col('TIPO') == 'DOCUMENTALES'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("DOCUMENTALES")
        , max(when((col('TIPO') == 'DOCUMENTALES'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_DOCUMENTALES")
        , max(when((col('TIPO') == 'DOCUMENTALES'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_DOCUMENTALES")

        , max(when((col('TIPO') == 'HBO'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("HBO")
        , max(when((col('TIPO') == 'HBO'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_HBO")
        , max(when((col('TIPO') == 'HBO'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_HBO")

        , max(when((col('TIPO') == 'PROMO_HBO'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("PROMO_HBO")
        , max(when((col('TIPO') == 'PROMO_HBO'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_PROMO_HBO")
        , max(when((col('TIPO') == 'PROMO_HBO'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_PROMO_HBO")

        , max(when((col('TIPO') == 'FILMIN'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("FILMIN")
        , max(when((col('TIPO') == 'FILMIN'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_FILMIN")
        , max(when((col('TIPO') == 'FILMIN'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_FILMIN")

        , max(when((col('TIPO') == 'PROMO_FILMIN'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("PROMO_FILMIN")
        , max(when((col('TIPO') == 'PROMO_FILMIN'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_PROMO_FILMIN")
        , max(when((col('TIPO') == 'PROMO_FILMIN'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_PROMO_FILMIN")

        #VODAFONE PASS
        , max(when((col('TIPO') == 'VIDEOHDPASS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("VIDEOHDPASS")
        , max(when((col('TIPO') == 'VIDEOHDPASS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_VIDEOHDPASS")
        , max(when((col('TIPO') == 'VIDEOHDPASS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_VIDEOHDPASS")

        , max(when((col('TIPO') == 'MUSICPASS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("MUSICPASS")
        , max(when((col('TIPO') == 'MUSICPASS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_MUSICPASS")
        , max(when((col('TIPO') == 'MUSICPASS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_MUSICPASS")

        , max(when((col('TIPO') == 'VIDEOPASS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("VIDEOPASS")
        , max(when((col('TIPO') == 'VIDEOPASS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_VIDEOPASS")
        , max(when((col('TIPO') == 'VIDEOPASS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_VIDEOPASS")

        , max(when((col('TIPO') == 'SOCIALPASS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("SOCIALPASS")
        , max(when((col('TIPO') == 'SOCIALPASS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_SOCIALPASS")
        , max(when((col('TIPO') == 'SOCIALPASS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_SOCIALPASS")

        , max(when((col('TIPO') == 'MAPSPASS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("MAPSPASS")
        , max(when((col('TIPO') == 'MAPSPASS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_MAPSPASS")
        , max(when((col('TIPO') == 'MAPSPASS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_MAPSPASS")

        , max(when((col('TIPO') == 'CHATPASS'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("CHATPASS")
        , max(when((col('TIPO') == 'CHATPASS'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_CHATPASS")
        , max(when((col('TIPO') == 'CHATPASS'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_CHATPASS")

        , max(when((col('TIPO') == 'AMAZON'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("AMAZON")
        , max(when((col('TIPO') == 'AMAZON'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_AMAZON")
        , max(when((col('TIPO') == 'AMAZON'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_AMAZON")

        , max(when((col('TIPO') == 'PROMO_AMAZON'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("PROMO_AMAZON")
        , max(when((col('TIPO') == 'PROMO_AMAZON'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_PROMO_AMAZON")
        , max(when((col('TIPO') == 'PROMO_AMAZON'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_PROMO_AMAZON")

        , max(when((col('TIPO') == 'TIDAL'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("TIDAL")
        , max(when((col('TIPO') == 'TIDAL'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_TIDAL")
        , max(when((col('TIPO') == 'TIDAL'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_TIDAL")

        , max(when((col('TIPO') == 'PROMO_TIDAL'), data_service_tmp1_basic.COD_SERVICIO)
              .otherwise(None)).alias("PROMO_TIDAL")
        , max(when((col('TIPO') == 'PROMO_TIDAL'), data_service_tmp1_basic.FECHA_INST)
              .otherwise(None)).alias("FX_PROMO_TIDAL")
        , max(when((col('TIPO') == 'PROMO_TIDAL'), data_service_tmp1_basic.SERV_PRICE)
              .otherwise(None)).alias("PRICE_PROMO_TIDAL")
        )
                      )

cols_tv_charges = ['PRICE_SRV_BASIC',\
'PRICE_TV_CUOT_CHARGES',\
'PRICE_TV_CUOTA_ALTA',\
'PRICE_DECO_TV',\
'PRICE_TV_TARIFF',\
'PRICE_TV_PROMO',\
'PRICE_TV_PROMO_USER',\
'PRICE_TV_LOYALTY',\
'PRICE_TV_SVA',\
'PRICE_TV_ABONOS',\
'PRICE_TRYBUY_AUTOM_TV',\
'PRICE_TRYBUY_TV',\
'PRICE_ZAPPER_TV',\
'PRICE_PVR_TV',\
'PRICE_MOTOR_TV',\
'PRICE_FOOTBALL_TV',\
'PRICE_CINE',\
'PRICE_SERIES',\
'PRICE_SERIEFANS',\
'PRICE_SERIELOVERS',\
'PRICE_CINEFANS',\
'PRICE_PEQUES',\
'PRICE_DOCUMENTALES']

cols_mobile_charges = ['PRICE_SRV_BASIC',\
'PRICE_TARIFF',\
'PRICE_DTO_LEV1',\
'PRICE_DTO_LEV2',\
'PRICE_DTO_LEV3',\
'PRICE_DATA',\
'PRICE_VOICE_TARIFF',\
'PRICE_DATA_ADDITIONAL',\
'PRICE_OOB',\
'PRICE_NETFLIX_NAPSTER',\
'PRICE_ROAM_USA_EUR',\
'PRICE_ROAMING_BASIC',\
'PRICE_ROAM_ZONA_2',\
'PRICE_CONSUM_MIN']

data_service_tmp3_basic = (data_service_tmp2_basic
                       .withColumn('TV_TOTAL_CHARGES_PREV', sum(coalesce(data_service_tmp2_basic[c], lit(0)) for c in cols_tv_charges))
                       .withColumn('MOBILE_BAM_TOTAL_CHARGES_PREV', sum(coalesce(data_service_tmp2_basic[c], lit(0)) for c in cols_mobile_charges))
                      )

w_srv_2 = Window().partitionBy("NUM_CLIENTE", "MSISDN").orderBy(desc("TACADA"))

data_service_tmp4_basic = (data_service_tmp3_basic
                       .withColumn("rowNum", row_number().over(w_srv_2))
                       .where(col('rowNum') == 1)
                      )

w_srv_3 = Window().partitionBy("MSISDN").orderBy(desc("TACADA"))

data_service_tmp5_basic = (data_service_tmp4_basic
                       .withColumn("rowNum", row_number().over(w_srv_3))
                       .where(col('rowNum') == 1)
                       .join(data_mapper, (col("OBJID") == col("CAMPO3")), 'leftouter')
                      )

data_service_tmp6_basic = (data_service_tmp5_basic
                       .withColumn('flag_msisdn_err',
                                   when(col('msisdn').like('% '), 1)
                                   .otherwise(0))
                       .withColumn('msisdn', trim(col('msisdn')))
                       .where(col('msisdn').isNotNull())
                       .where(col('msisdn') != '')
                       .withColumn('TV_TOTAL_CHARGES',
                                   when(col('rgu') == 'tv', data_service_tmp5_basic.TV_TOTAL_CHARGES_PREV)
                                   .otherwise(0))
                       .withColumn('MOBILE_BAM_TOTAL_CHARGES',
                                   when(col('rgu').isin('bam', 'bam_mobile', 'mobile'),data_service_tmp5_basic.MOBILE_BAM_TOTAL_CHARGES_PREV)
                                   .otherwise(0))
                       .drop(col('TV_TOTAL_CHARGES_PREV'))
                       .drop(col('MOBILE_BAM_TOTAL_CHARGES_PREV'))
                       .drop(col('rowNum'))
                      )

data_service_tmp6_basic = data_service_tmp6_basic.filter((~isnull(col('NUM_CLIENTE'))) & (~col('NUM_CLIENTE').isin('', ' ')))

Getting Services Information...


In [25]:
#df_service.where(col("OBJID").isin("722163038","945388489","936853265")).join(spark.read.load("/data/udf/vf_es/amdocs_inf_dataset/customer/year=2019/month=9/day=30"), on=["NUM_CLIENTE"], how="inner").show()
from churn.analysis.triggers.base_utils.base_utils import get_customers, get_customers_insights

df_c = get_customers_insights(spark, "20190930", add_columns=["TIPO_DOCUMENTO", "X_CLIENTE_PRUEBA", "NIF_CLIENTE"])
# df_c.where(col("NIF_CLIENTE").rlike("^999")).groupby("TIPO_DOCUMENTO").agg(sql_count("*")).show()

df_c.where(col("NUM_CLIENTE")=="954345290").show()


+-----------+--------------+-----------+---------------------------+----------------+-----------+------------------+---------------+-------------------+
|SUPEROFERTA|TIPO_DOCUMENTO|NUM_CLIENTE|CLASE_CLI_COD_CLASE_CLIENTE|X_CLIENTE_PRUEBA|NIF_CLIENTE|COD_ESTADO_GENERAL|NIF_FACTURACION|    FECHA_MIGRACION|
+-----------+--------------+-----------+---------------------------+----------------+-----------+------------------+---------------+-------------------+
|       ON19|        N.I.F.|  954345290|                         RS|               0|  04612449I|                09|      79075358R|1753-01-01 00:10:04|
+-----------+--------------+-----------+---------------------------+----------------+-----------+------------------+---------------+-------------------+



In [43]:
from churn.analysis.triggers.base_utils.base_utils import get_services

df_serv = get_services(spark, "20190930")

df_serv.where(col("NUM_CLIENTE")=="954345290").show()

+-----------+-----------+------+---------+--------------+-----+------+------------+---------------+---+--------+----+------+---------+-----------+------------+------------+---------------+------------------+----+-------+----------+--------+-----------+--------------+--------+-----------+--------------+--------+-----------+--------------+---------------+------------------+---------------------+---+------+---------+---------------+------------------+---------------------+-------------+----------------+-------------------+------------+---------------+------------------+-----------+--------------+-----------------+----------+-------------+----------------+------+--------+-----------+--------------+---------------+-----------+--------------+-----------------+-------+----------+-------------+-----------------+-------------+-------------+----------------+-------------------+---------+------------+---------------+---------------+------------------+---------------------+--------+-----------+---

In [112]:
df_serv.where(col("MSISDN")=="600657317").show()

KeyboardInterrupt: 

In [29]:
from churn.analysis.triggers.base_utils.base_utils import get_customers, get_active_services_insights

df_active_services = get_active_services_insights(spark, "20190930", customer_cols=None, service_cols=["TARIFF", "OBJID", "campo2", "Instancia_P"])#.where(col("rgu")=="mobile")
#df_active_services = df_active_services.where( ( (col("TARIFF").isNull()) | ((~col("TARIFF").isin("DTVC2", "DTVC5")))))

df_active_services.where(col("NUM_CLIENTE")=="954345290").show()

#df_active_services.count()

+---+-----------+------+---------------------------+------+------+-----------+-----+-----------+------------------+---------+
|rgu|num_cliente|TARIFF|clase_cli_cod_clase_cliente|msisdn|campo2|Instancia_P|OBJID|nif_cliente|cod_estado_general|srv_basic|
+---+-----------+------+---------------------------+------+------+-----------+-----+-----------+------------------+---------+
+---+-----------+------+---------------------------+------+------+-----------+-----+-----------+------------------+---------+



In [27]:
#from churn_nrt.src.data.customer_base import CustomerBase
from churn_nrt.src.data_utils.base_filters import get_mobile_base
df_bd = get_mobile_base(spark, date_="20190930", save=False)
df_bd.where(col("NUM_CLIENTE")=="954345290").show()
#df_bd.count()

[DataTemplate] module name = customer_base | path = /data/udf/vf_es/churn_nrt/customer_base | verbose = True
[DataTemplate] get_module | Asking module customer_base - for closing_day=20190930 save=False save_others=False
[DataTemplate] get_module | args: () | kwargs: {}
[CustomerBase] is_default_module | args: () | kwargs: {}
[DataTemplate] get_module | Found already an existing module - '/data/udf/vf_es/churn_nrt/customer_base/year=2019/month=9/day=30'
+-----------+-----------+------+---+------------------+---------+------+-----------+-------------------+----------------------+------------------+-----------------------+--------------------------+---------------------+-------------------+-----------+---------+-----------------------+----------------+
|nif_cliente|NUM_CLIENTE|msisdn|rgu|cod_estado_general|srv_basic|TARIFF|segment_nif|nb_fbb_services_nif|nb_mobile_services_nif|nb_tv_services_nif|nb_prepaid_services_nif|nb_bam_mobile_services_nif|nb_fixed_services_nif|nb_bam_services_nif|

In [ ]:
data_service_tmp1_basic.where( col("NUM_CLIENTE")=="954345290").show()

In [ ]:
data_service_tmp2_basic.where( col("NUM_CLIENTE")=="954345290").show()

In [ ]:
data_service_tmp4_basic.where( col("NUM_CLIENTE")=="954345290").show()

In [ ]:
data_service_tmp6_basic.where( col("MSISDN") == "600392900").show()

In [ ]:
(spark.read.load("/data/raw/vf_es/customerprofilecar/SERVICESOW/1.1/parquet/").where(concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day)).where(col("num_serie") == "600657317").show()

In [113]:
(spark.read.load("/data/raw/vf_es/customerprofilecar/SERVICESOW/1.0/parquet/").where(concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day)).where(col("NUM_CLIENTE") == "954345290").show()

+-----+-----------+------------+---------+--------+-------+----------+----------+-----------+---------------+--------------+----------------+---------------+--------------+--------------+-----------+---------+------------+----------+---------+------------+--------+--------+---------+----------+--------+-------+-----------------+-----------------+--------+-------------+------------------------+---+--------------------+--------------+---------+---------+-----------------+--------+--------+---------+-------+-----------+------------------+---------------+---------------+---------------+-----------------+-----+------+----------+-------------+--------------------+---------------+----+-----+---+
|OBJID|NUM_CLIENTE|COD_SERVICIO|INSTANCIA|NUM_PRIM|NUM_ADD|FECHA_INST|FECHA_CAMB|CAMP_ACTIVE|OPE_DIVISION_ID|OPE_EMPRESA_ID|OPE_OPERADORA_ID|ESTADO_SERVICIO|FECHA_INI_CAMP|FECHA_FIN_CAMP|TIPO_PRECIO|PRIM_RATE|PRICE_METHOD|CHARGE_PER|REL_OBJID|PART_ESTATUS|FILLER_1|SERVICIO|NUM_SERIE|PRICE_CODE|RGU_REA

In [48]:
(spark.read.load("/data/raw/vf_es/customerprofilecar/SERVICESOW/1.1/parquet/").where(concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day)).where(col("NUM_CLIENTE") == "954345290").show()

+-----------+-----------+------------+----------+--------+-------+-------------------+-------------------+-----------+---------------+--------------+----------------+---------------+-------------------+-------------------+-----------+--------------------+------------+----------+-----------+--------------------+--------+--------+---------+----------+--------------------+--------------------+-------------------+-------------------+--------+-------------+------------------------+---+--------------------+--------------+---------+--------------------+-----------------+--------+--------+---------+-------+-----------+------------------+---------------+---------------+---------------+-------------------+-----+------+----------+--------------------+--------+-----------+--------------+-----------+----------------+------------+-----------------+---------------+----------------+--------------+---------------------+-------------------+--------------------+--------------------+----+-----+---+
|     

In [ ]:
closing_day = "20190930"

data_service_ori = (spark.read.load("/data/raw/vf_es/customerprofilecar/SERVICESOW/1.1/parquet/").where(concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day))

#data_serviceprice_ori = (spark.read.load("/data/raw/vf_es/priceplanstariffs/PLANPRIC_ONO/1.0/parquet/").where(concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day))
w_srv = Window().partitionBy("OBJID").orderBy(desc("year"), desc("month"), desc("day"))
data_service_ori_norm = (data_service_ori
                         .withColumn("rowNum", row_number().over(w_srv))
                         .where(col('rowNum') == 1))
(data_service_ori_norm.withColumn("Instancia_P", trim(split(data_service_ori_norm.INSTANCIA, '\\.')[0]))
.withColumn("Instancia_S", split(data_service_ori_norm.INSTANCIA, '\\.')[1])
.withColumn('MSISDN',
                                       when((col('Instancia_S').isNull() & (col('COD_SERVICIO') == 'TVOTG')),
                                            concat(lit('FICT_TVOTG_'), data_service_ori_norm.NUM_CLIENTE))
                                       .when((col('Instancia_S').isNull() & (col('COD_SERVICIO') != 'TVOTG')),
                                             data_service_ori_norm.NUM_SERIE)
                                       .otherwise(lit(None)))).where(col("MSISDN") == "600657317").select("NUM_CLIENTE", "NUM_SERIE", "MSISDN", "Instancia_P", "Instancia_S", "COD_SERVICIO", "objid").show()

In [54]:
LOC_RT_PATH = '/data/udf/vf_es/ref_tables/amdocs_ids/'
LOC_RT_EXPORT_MAP = LOC_RT_PATH + 'RBL_EXPORT_MAP.TXT'
LOC_RT_PARAM_OW_SERVICES= LOC_RT_PATH + 'PARAM_OW_SERVICES.TXT'
data_service_param = spark.read.csv(LOC_RT_PARAM_OW_SERVICES, header=True,sep='\t')
#data_service_param = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter='\t').load('/tmp/rbuendi1/PARAM_AMD_DLAB/')
data_service_param = (data_service_param.where(col('rgu').isNotNull())
  .withColumn('rgu', 
    when(data_service_param['rgu'] == 'bam-movil', 'bam_mobile')
    .when(data_service_param['rgu'] == 'movil', 'mobile')
    .otherwise(data_service_param['rgu'])))

data_service_param.where(col("COD_SERVICIO").isin("M100C", "SECUR")).show()

+------------+---+----------+-------+----+---------------+------+-------------+
|COD_SERVICIO|rgu|RGU_mobile|RGU_BAM|TIPO|PRIMERAS_LINEAS|LEGACY|DESC_SERVICIO|
+------------+---+----------+-------+----+---------------+------+-------------+
+------------+---+----------+-------+----+---------------+------+-------------+



In [61]:
data_service_param.where(col("COD_SERVICIO").isin("MRSUI")).show(100)

+------------+------+----------+-------+-----------+---------------+------+--------------------+
|COD_SERVICIO|   rgu|RGU_mobile|RGU_BAM|       TIPO|PRIMERAS_LINEAS|LEGACY|       DESC_SERVICIO|
+------------+------+----------+-------+-----------+---------------+------+--------------------+
|       MRSUI|mobile|         1|      0|SERV_BASICO|              0|     1|Susp. Impago móvi...|
+------------+------+----------+-------+-----------+---------------+------+--------------------+



In [59]:
spark.read.load("/data/raw/vf_es/customerprofilecar/WEBSERVICES/1.0/parquet").where(col("COD_SERVICIO").isin("M100C")).show(100)

+------------+---+---------------+-------------+--------+----+----------+-----+---------+------+--------------+-----------------+------------+--------+--------+--------+--------+--------+--------+------------+--------+---------+--------+-------+------+-------------+---------------+-------------------------+----------+------+---------+---------+---------+--------------------+---------------+----+-----+---+
|COD_SERVICIO|RGU|PARAMETRIZACION|DESC_SERVICIO|PRODUCTO|PESO|REPETICION|CLASE|LIN_EQUIV|AMBITO|AFECTA_PAQUETE|AFECTA_MOVIMIENTO|MEGAPRODUCTO|DETALLE1|DETALLE2|DETALLE3|DETALLE4|DETALLE5|DETALLE6|FORMULA_GGCC|RGF_GGCC|TIPO_PLAN|CONTABLE|TRATADO|TARIFA|OBSERVACIONES|USUARIO_ULT_MOD|FECHA_ULTIMA_MODIFICACION|SUSPENSION|IMPAGO|FAMILY_VF|X_VF_TYPE|X_SUBTIPO|service_processed_at|service_file_id|year|month|day|
+------------+---+---------------+-------------+--------+----+----------+-----+---------+------+--------------+-----------------+------------+--------+--------+--------+--------+----

In [69]:
df_tr = spark.read.load("/data/udf/vf_es/churn_nrt/navcomp/15/year=2020/month=1/day=2")
from churn_nrt.src.utils.pyspark_utils import count_nans

A = count_nans(df_tr)

In [82]:
from churn_nrt.src.data.orders_sla import OrdersSLA
df_metadata = OrdersSLA(spark).get_metadata()



metadata_cols = df_metadata.rdd.map(lambda x: x['feature']).collect()



[DataTemplate] module name = ccc/msisdn | path = /data/udf/vf_es/churn_nrt/ccc/msisdn | verbose = True
[CCC] get_bucket_info | Num entries in bucket: 126855
[CCC] get_bucket_info | Elapsed time 1.6561601162 secs 


In [105]:
module_name = "orders_sla"

if module_name == "orders_sla":
    from churn_nrt.src.data.orders_sla import OrdersSLA
    df_metadata = OrdersSLA(spark).get_metadata()
elif module_name == "customer":
    from churn_nrt.src.data.customer_data import Customer
    df_metadata = Customer(spark).get_metadata()
elif module_name == "service":
    from churn_nrt.src.data.service_data import Service
    df_metadata = Service(spark).get_metadata()
    
    
ids = ["msisdn", "NUM_CLIENTE", "NIF_CLIENTE"]

metadata_cols = df_metadata.rdd.map(lambda x: x['feature']).collect()


module_partitions = spark.read.load("/data/udf/vf_es/churn_nrt/{}/".format(module_name)).select("year", "month", "day").distinct().rdd.map(lambda x: (x['year'], x['month'], x['day'])).collect()

for part in module_partitions:
    
    path_module = "/data/udf/vf_es/churn_nrt/orders_sla/year={}/month={}/day={}".format(part[0], part[1], part[2])
    module_cols = list(set(spark.read.load(path_module).columns) - set(ids))
    
    
    A = list(set(metadata_cols) - set(module_cols))
    B = list(set(module_cols) - set(metadata_cols))
    
    if A or B:
        print("ERROR partition '{}'".format(path_module))
        
        if A:
            print("\t metadata has columns that are not present in module: {}".format(",".join(A)))
        if B:
            print("\t module has columns that are not present in metadata: {}".format(",".join(B)))
        
    else:
        print("OK partition '{}'".format(path_module))



[DataTemplate] module name = orders_sla | path = /data/udf/vf_es/churn_nrt/orders_sla | verbose = True
OK partition '/data/udf/vf_es/churn_nrt/orders_sla/year=2019/month=10/day=31'


In [83]:
list(set(metadata_cols) - set(module_cols))

[]

In [85]:
[col_ for col_ in metadata_cols if col_ == "inc_OTHER_CUSTOMER_INFORMATION_MANAGEMENT_w4vsw4"]

[u'inc_OTHER_CUSTOMER_INFORMATION_MANAGEMENT_w4vsw4']

In [84]:
list(set(module_cols) - set(metadata_cols))

['msisdn']

In [96]:
spark.read.load("/data/udf/vf_es/churn_nrt/customer_base/year=2020/month=1/day=2").select("segment_nif").groupby("segment_nif").agg(sql_count("*")).show()

+-----------+--------+
|segment_nif|count(1)|
+-----------+--------+
|      Other|       3|
|Mobile_only| 6002009|
+-----------+--------+

